# **Projeto Final do Curso de pós-graduação BI-Master da PUC-RIO**

Aluno: Ronald Alzamende Martins




# Montagem do drive:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Direcionando para uma pasta dentro do drive:

In [ ]:
if (True):
  import os
  os.chdir("drive/MyDrive/TCC")  #MAPEAR PARA O LOCAL NO DRIVE ONDE os arquivos serão armazenados

In [ ]:
!ls

# **PRIMEIRA ETAPA - Coleta de dados**

Os dados utilizados neste trabalho são dados públicos, publicados semanalmente pelo Instituto Nacional da Propriedade Industrial (INPI) através da RPI - Revista da Propriedade Industrial, no website da instituição:
http://revistas.inpi.gov.br/rpi/. 

Atualmente, a RPI é publicada todas as terças-feiras. Em cada RPI há despachos relativos somente aos pedidos de patente de invenção ou de modelo de utilidade. Estes despachos serão explicados com mais detalhes na próxima etapa.

O nosso objetivo, neste trabalho, será o de formar dataframes a partir da extração das informações contidas nos despachos das RPI's, e então poder fazer algumas análises dos dados extraídos.

Antecipadamente, sabemos que, no período que serão extraídas as rpi's, o número total de rpi's é de 1453, isto é, no final da coleta, teremos que ter 1453 arquivos .txt!

O download de cada RPI do site será feito utilizando-se as bibliotecas Selenium e BeautifulSoup 



Instalando as bibliotecas necessárias:

In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver

In [ ]:
!wget https://chromedriver.storage.googleapis.com/2.42/chromedriver_linux64.zip  && unzip chromedriver_linux64

In [ ]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--start-maximized')
chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36')
chrome_options.add_argument('--ignore-certificate-errors')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options, service_args=['--verbose', '--log-path=/tmp/chromedriver.log'])

In [ ]:
# Importando a biblioteca beautifulsoup
from bs4 import BeautifulSoup
import time
import subprocess

In [ ]:
# Acessando a URL
wd.get("http://revistas.inpi.gov.br/rpi/")

Uma das vantagens de se utilizar a biblioteca selenium é a facilidade em simular as ações de um usuário ao acessar determinada url, sendo uma delas a de dar um click em determinado local e digitar uma informação necessária.

Vamos verificar o código, passo a passo, vamos usar o período 01/01/1993 a 10/01/1993 para testar se o código está funcionando:

In [ ]:
# 1º ação do usuário na página, clicar no campo "Buscar Patentes"
busca_patente = wd.find_element_by_xpath('//a[@id="6"]')
busca_patente.click()

In [ ]:
# 2º ação do usuário na página, clicar no tipo de pesquisa "por data"
tipo_pesquisa = wd.find_element_by_xpath('//input[@value="2"]')
tipo_pesquisa.click()

In [ ]:
# 3º ação do usuário na página, entrar com a data inicial de publicação da rpi
data_inicial = wd.find_element_by_xpath('//input[@id="dataInicial"]')
data_inicial.click()
print("Digite a data inicial no formato xx/xx/xxxx e tecle enter")
inicial=input() # utilizando o método input para entrar com a data inicial
data_inicial.send_keys(inicial)

In [ ]:
# 4º ação do usuário na página, entrar com a data final de publicação da rpi
data_final = wd.find_element_by_xpath('//input[@id="dataFinal"]')
data_final.click()
print("Digite a data final no formato xx/xx/xxxx, e tecle enter")
final=input() # utilizando o método input para entrar com a data final
data_final.send_keys(final)


In [ ]:
# 5º ação do usuário, clicar no botão Buscar
button = wd.find_elements_by_xpath('//button[@class="btn rst"]')[1]
button.click()


In [ ]:
# E, finalmente, extraindo o conteúdo da busca que aparece no site da rpi
result = wd.find_element_by_xpath('//tbody[@id="resultado"]')
result_rpi=result.get_attribute('innerHTML')

In [ ]:
# Visualizando o que é extraído da página:
result_rpi

In [ ]:
# Utilizando o beautifulsoup para traduzir a linguagem html
rpi_page_soup = BeautifulSoup(result_rpi,'html.parser')
rpi_page_soup

In [ ]:
# Pegando todos os arquivos 
page_links_url = [a['href'] for a in rpi_page_soup.find_all('a', href=True)]
page_links_url

In [ ]:
# Pegando os arquivos somente com extensão .zip
page_zip_urls = [zip_url for zip_url in page_links_url if ".zip" in zip_url]
page_zip_urls

In [ ]:
# Verificando o tamanho da lista
len(page_zip_urls)

Agora vamos fazer o download das revistas:

In [ ]:
import subprocess

for url in page_zip_urls:
  print("Extraindo o pdf da url: ", url)
  getRpm = 'wget --no-check-certificate -P ./ %s' %url
  subprocess.run(getRpm, shell=True)

print("Processo finalizado!")

O código funcionou, passo a passo,  para extrair as revistas no período de 01/01/1993 a 10/01/1993. Vamos juntar as etapas, excluir a revista já extraída e extrair as RPI's no período de 01/01/1993 a 08/12/2020.

In [ ]:
# Para cada ação, vamos inserir um tempo de espera (time.sleep()) no código para a ação poder ser realizada:
import time

# Acessando a URL
wd.get("http://revistas.inpi.gov.br/rpi/")
time.sleep(2)

# 1º ação do usuário na página, clicar no campo "Buscar Patentes"
busca_patente = wd.find_element_by_xpath('//a[@id="6"]')
busca_patente.click()
time.sleep(2)

# 2º ação do usuário na página, clicar no tipo de pesquisa "por data"
tipo_pesquisa = wd.find_element_by_xpath('//input[@value="2"]')
tipo_pesquisa.click()
time.sleep(2)

# 3º ação do usuário na página, entrar com a data inicial de publicação da rpi
data_inicial = wd.find_element_by_xpath('//input[@id="dataInicial"]')
data_inicial.click()
print("Digite a data inicial no formato xx/xx/xxxx e tecle enter")
inicial=input() # utilizando o método input para entrar com a data inicial
data_inicial.send_keys(inicial)
time.sleep(2)

# 4º ação do usuário na página, entrar com a data final de publicação da rpi
data_final = wd.find_element_by_xpath('//input[@id="dataFinal"]')
data_final.click()
print("Digite a data final no formato xx/xx/xxxx, e tecle enter")
final=input() # utilizando o método input para entrar com a data final
data_final.send_keys(final)
time.sleep(2)

# 5º ação do usuário, clicar no botão Buscar
button = wd.find_elements_by_xpath('//button[@class="btn rst"]')[1]
button.click()
time.sleep(4)

# E, finalmente, extraindo o conteúdo da busca que aparece no site da rpi
result = wd.find_element_by_xpath('//tbody[@id="resultado"]')
result_rpi=result.get_attribute('innerHTML')
time.sleep(3)

# Utilizando o beautifulsoup para traduzir a linguagem html
rpi_page_soup = BeautifulSoup(result_rpi,'html.parser')
time.sleep(3)

# Pegando todos os arquivos 
page_links_url = [a['href'] for a in rpi_page_soup.find_all('a', href=True)]
time.sleep(3)

# Pegando os arquivos somente com extensão .zip
page_zip_urls = [zip_url for zip_url in page_links_url if ".zip" in zip_url]
time.sleep(3)

In [ ]:
len(page_zip_urls)

Há um arquivo repetido, veremos adiante.

In [ ]:
# Importando as revistas
from datetime import datetime
start_time = datetime.now()
for url in page_zip_urls:
  print("Extraindo o pdf da url: ", url)
  getRpm = 'wget --no-check-certificate -P ./ %s' %url
  subprocess.run(getRpm, shell=True)
end_time = datetime.now()
print('\n')
print('Tempo total de importação das revistas: {}'.format(end_time - start_time))
print("Processo finalizado!")

Verificando o que temos no drive:

In [ ]:
!ls

In [ ]:
# Como não faremos mais atividades de scrapping, vamos excluir os arquivos chromedriver, chromedriver_linux64.zip, bem como o arquivo repetido P2130.zip.1
import os
os.remove('chromedriver')
os.remove('chromedriver_linux64.zip')
os.remove('P2130.zip.1')

In [ ]:
!ls

In [ ]:
import os 
os.chdir('../')

In [ ]:
!ls

Após o download de todas as rpi's no formato txt, vamos agora "unzip" todos os arquivos:

In [ ]:
# Função para "unzip" os arquivos zipados
import zipfile
def un_zipFiles(path):
    files=os.listdir(path)
    for file in files:
        if file.endswith('.zip'):
            filePath=path+'/'+file
            zip_file = zipfile.ZipFile(filePath)
            for names in zip_file.namelist():
                zip_file.extract(names,path)
            zip_file.close() 

In [ ]:
# Chamando a função:
path="TCC"
un_zipFiles(path)

In [ ]:
os.chdir("TCC")
!ls

Vamos deletar os arquivos .zip e .xml que ficaram na pasta

In [ ]:
os.chdir('../')
dir_name = "TCC"
test = os.listdir(dir_name)

for item in test:
    if item.endswith(".zip") or item.endswith(".xml"):
        os.remove(os.path.join(dir_name, item))

In [ ]:
!ls

In [ ]:
os.chdir("TCC")
!ls

In [ ]:
# Contando o número de arquivos
import glob
j=0
for x in glob.glob('*'):
  if x.endswith(".txt") or x.endswith(".TXT"):
   j=j+1
j

Portanto, até aqui tudo ok! Temos ao todo 1453 rpi's no formato .txt!

Encerramos a primeira parte do trabalho, que foi a coleta dos dados, vamos à segunda etapa!

# **SEGUNDA ETAPA: ETL - Extraction, Transformation e Loading (extração, transformação e carregamento dos dados)**


Nesta etapa, após o download de todas as RPI's, no período de 01/01/1993 a 08/12/2020, vamos extrair as informações contidas em alguns despachos, para a construção dos dataframes que serão utilizados nas Etapas seguintes. 

Estes dataframes serão detalhados mais adiante.



# **1.   Extração**


Vamos visualizar o que há nos arquivos txt:

In [ ]:
import pandas as pd

In [ ]:
# Tentando abrir com encoding utf-8
# Abrindo o arquivo P1411  
P1411 = open("P1411.txt", encoding = "utf-8")
print(P1411.read())
P1411.close()

Com o encoding utf-8 deu erro, vamos tentar abrir com o encoding iso-8859-1:

In [ ]:
P1411 = open("P1411.txt", encoding = "iso-8859-1")
print(P1411.read())
P1411.close()

In [ ]:
# Abrindo o arquivo P2600
P2600 = open("P2600.txt", encoding = "utf-8")
print(P2600.read())
P2600.close()

Vimos que os arquivos podem abrir ou com o encoding utf-8 ou com o iso-8859-1.

Com base na estrutura sequencial de cada despacho, construíremos então três dataframes: 


1.   Dataframe com os Despachos 1.3 (PCT), 3.1 (Não PCT) e 3.2. O despacho 1.3 trata da publicação da entrada na fase nacional do pedido internacional depositado através do Tratado de Cooperação de Patentes - PCT, o despacho 3.1 refere-se à Publicação do Pedido de Patente ou de Certificado de Adição de Invenção e o despacho 3.2 informa a Publicação antecipada do pedido depositado, a requerimento do depositante. Neste dataframe, em cada coluna, teremos as seguintes informações:

- idpedido: é o número do pedido de patente (21);

- data_deposito: é a data do depósito (22) do pedido;

- ipc: é a Classificação Internacional de Patentes (IPC, na sigla em inglês) (51);

- titulo: é o Título (54) do pedido de patente;  

- depositante: é o Depositante (71) do pedido de patente, podendo ser o inventor ou uma empresa. Esta informação também contém o país de origem do depositante, entre parênteses, que será desmembrada em outra coluna; 

- despacho: informando se o pedido teve a publicação 1.3, 3.1 ou 3.2; e

- rpi: informa em qual revista foi publicado o despacho e a data da publicação da revista.


2.   Um dataframe com os despachos 6.1, 6.20, 6.21, 6.22, 7.1 (os despachos com início 6 tratam de exigências formais/técnicas, a qual indica que há uma propenção a deferir o pedido de patente, isto é, o pedido tem chance de ter a patente concedida, e o 7.1 que é uma ciência a qual indica que há uma propenção a indeferir o pedido de patente (9.2), ou seja, o pedido pode não ter a concesssão da patente requerida). Em cada coluna, deverá conter as seguintes informações:

- idpedido: é o número do pedido de patente (21);

- data_deposito: é a data do depósito (22) do pedido;

- depositante: é o Depositante (71) do pedido de patente, podendo ser o inventor ou uma empresa. Esta informação também contém o país de origem do depositante, entre parênteses, que será desmembrada em outra coluna; 

- despacho: informando os despachos; e

- rpi: informa em qual revista foi publicado o despacho e a data da publicação da revista.


3.   E, outro dataframe com os despachos 9.1, 9.2 e 11.2 (o 9.1 é a concessão da patente, o 9.2 é a não concessão da patente e o 11.2 é o arquivamento definitivo do pedido caso o depositante não responda à exigência/ciência no prazo de 90 dias). Neste dataframe, em cada coluna teremos as seguintes informações: 

- idpedido: é o número do pedido de patente (21);

- data_deposito: é a data do depósito (22) do pedido;

- titulo: é o Título (54) do pedido de patente;  

- depositante: é o Depositante (71) do pedido de patente, podendo ser o inventor ou uma empresa. Esta informação também contém o país de origem do depositante, entre parênteses, que será desmembrada em outra coluna; 

- despacho: informando os despachos; e

- rpi: informa em qual revista foi publicado o despacho e a data da publicação da revista.

In [ ]:
# Verificando o tipo da variável P2600
type(P2600)

In [ ]:
P1411

In [ ]:
P2600

In [ ]:
P2600[0]

O tipo da variável P1411 ou P2600, que abriu o arquivo txt, é _io.TextIOWrapper, um arquivo objeto que, segundo a mensagem de erro acima, não possui índice, portanto não é possível iterar sob esta variável. Vamos então trasformar o arquivo em uma lista, a qual é possível a manipulação dos dados (linhas). 

Para tanto, vamos criar uma função para abrir o arquivo com encoding utf-8 ou com o iso-8859-1 e retornar o arquivo já transformado em uma lista:

In [ ]:
# Vamos definir uma função para abrir os arquivos txt retornando o mesmo transformado em uma lista:
# Nos testes acima, verifcou-se que alguns arquivos txt abrem com o encoding utf-8 e outros com o iso-8859-1
def txtplista(t):
 try:
   with open(t, encoding = "utf-8") as f:
     list1 = [line.rstrip('\n') for line in f]
 except:
   with open(t, encoding = "ISO-8859-1") as f:
     list1 = [line.rstrip('\n') for line in f]

 f.close()
 return list1


In [ ]:
lista1=txtplista("P1411.txt")

In [ ]:
lista1

In [ ]:
len(lista1)

Observa-se que somente o arquivo P1513 possui 8554 linhas, esta revista é a mais antiga rpi que temos em nossa pasta! Vamos ver quantas linhas há na revista mais recente, a P2600. 

In [ ]:
lista2=txtplista("P2600.txt")
len(lista2)

In [ ]:
lista2

In [ ]:
lista2[0][0:1]

Vamos conhecer um pouco mais sobre os arquivos txt que extraímos:

In [ ]:
import glob

l=1
linhas=[]
for x in glob.glob("*"):
  if x.endswith(".txt") or x.endswith(".TXT"):
    lista=txtplista(x)
    nlinhas=len(lista)
    print('Nº de linhas da revista ',x,'=',nlinhas," arquivo:",l)
    linhas.append(nlinhas)
  l=l+1
print('\n') 
print("Processo de verificação Finalizado!")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
my_array = np.array(linhas)
my_array

In [ ]:
print('O número médio de linhas dos arquivos é: ', my_array.mean())

In [ ]:
print('A mediana de número de linhas dos arquivos é: ', np.median(my_array))

In [ ]:
print('O número máximo de linhas dos arquivos é: ', np.max(my_array))

In [ ]:
print('O número mínimo de linhas dos arquivos é: ', np.min(my_array))

In [ ]:
plt.hist(my_array, bins=20)
plt.show()

Vamos imprimir aleatoriamente 3 linhas de cada arquivo, para ter uma idéia do conteúdo de todos os arquivos.

In [ ]:
import glob
l=1
linhas=[]
for x in glob.glob("*"):
  lista=txtplista(x)
  print('RPI nº:',x," revista:",l)
  try:
    print(lista[0])
    print(lista[200])
    print(lista[5000])
  except:
    print(lista[0])
    print(lista[50])
    print(lista[130])
  l=l+1
  
print('\n') 
print("Processo de verificação Finalizado!")

Da verificação de todas as revistas acima, podemos identificar alguns padrões:

- Na primeira linha de cada arquivo (linha 0) está o número da revista e a data de publicação da revista.

- As linhas que possuem informações que serão extraídas possuem o formato (xx).

- Cada despacho de interesse, 1.3 por exemplo, tem o formato '(Cd) 1.3'.

E, como já verificado anteriormente, analisando dois arquivos completos, podemos observar a seguinte estrutura geral para os despachos 1.3, 3.1 e 3.2:

(Cd) despacho

(21) número do pedido de patente

(22) data do depósito

(51) ipc

(54) título

(71) depositante da patente + (país de residência do depositante)


No entanto, não é sempre que a ordem após o despacho apresenta esta sequência, alguns pedidos possuem prioridade unionista cujo código é (30), um dado que não fará parte da análise. Outro dado que as RPI's mais antigas possuem é o resumo (código (57)), que segue logo após a linha do título e, infelizmente, nas RPI's mais recentes o resumo foi excluído e, portanto, não será utilizado para a análise. Além disso, nas RPI's mais recentes, além da ipc, o pedido também apresenta mais uma linha, com o código (52), que é a clissificação cpc, outro dado que também não é do nosso interesse. Em alguns pedidos, pode não ter a classificação (ipc), pode não ter título e pode não ter o nome do depositante também. O que dificulta mais o desenvolvimento de um código para a extração.

Para outros despachos a estrutura é um pouco semelhante, mas com menos informações (linhas) sequenciais.

Ao escrever o código para extrair as informações acima, deveremos levar estas diferenças em consideração. 

Com base nas informações acima, vamos escrever os códigos de extração, dependendo do despacho publicado, e depois testá-los em três revistas escolhidas aleatoriamente:

In [ ]:
# função para extrair as informações dos códigos 1.3, 3.1 e 3.2
def pedidos(list1):
  
  # Criando listas vazias
  rpi=[]
  despacho1=[]
  idpedido =[] 
  data=[]
  ipc=[]
  titulo=[]
  depositante=[]

  
  i=0  # linha da lista que será extraída a informação

  
  # Extração propriamente dita
  for j in list1[:]:
      nrpi=0
      nrpi=list1[0]
  
      if list1[i][5:]  == '1.3':
        
        ndespacho1=0
        npedido =0
        ndata=0
        nipc=0
        ntitulo=0
        ndepositante=0

        ndespacho1=list1[i][5:]
        npedido =list1[i+1][5:]
        ndata=list1[i+2][5:]
      
           
        # Este loop dentro do despacho é para verificar se o valor correto está sendo extraído
        # Caso não encontre a informação, o valor entrará  na lista como definido acima
        k=i+1
        while list1[k][1:3] != 'Cd':
           if list1[k][1:3]=='21':
             npedido =list1[k][5:]
           elif list1[k][1:3]=='22':  
             ndata=list1[k][5:]
           elif list1[k][1:3]=='51':
             nipc=list1[k][5:]
           elif list1[k][1:3]=='54':
             ntitulo=list1[k][5:]
           elif list1[k][1:3]=='71':
             ndepositante=list1[k][5:]
           k=k+1

        # Caso a variável não seja encontrada ou seja vazia "", garantiremos que seja preenchida com 0          
        if nrpi=="":
           nrpi=0
        elif ndespacho1=="":
           ndespacho1=0
        elif npedido == "":
           npedido=0
        elif ndata == "":
           ndata=0
        elif nipc=="":
           nipc=0
        elif ntitulo=="":
           ntitulo=0
        elif ndepositante == "":
           ndepositante=0 
                  
        # append as listas
        rpi.append(nrpi)
        idpedido.append(npedido) 
        data.append(ndata) 
        ipc.append(nipc)
        titulo.append(ntitulo)
        depositante.append(ndepositante)
        despacho1.append(ndespacho1)
               

      elif list1[i][5:]  == '3.1' or list1[i][5:]  == '3.2':
        
        ndespacho1=0
        npedido =0
        ndata=0
        nipc=0
        ntitulo=0
        ndepositante=0

        ndespacho1=list1[i][5:]
        npedido =list1[i+1][5:]
        ndata=list1[i+2][5:]
                             
        # Este loop dentro do despacho é para verificar se o valor correto está sendo extraído
        # Caso não encontre, o valor entrará na lista como definido acima 
      
        k=i+1 
        while list1[k][1:3]!= 'Cd':
           if list1[k][1:3]=='21':
             npedido =list1[k][5:]
           elif list1[k][1:3]=='22':  
             ndata=list1[k][5:]
           elif list1[k][1:3]=='51':
             nipc=list1[k][5:]
           elif list1[k][1:3]=='54':
             ntitulo=list1[k][5:]
           elif list1[k][1:3]=='71':
             ndepositante=list1[k][5:]  
           k=k+1
          
        # Caso a variável não seja encontrada ou seja vazia "", garantiremos que seja preenchida com 0          
        if nrpi=="":
           nrpi=0
        elif ndespacho1=="":
           ndespacho1=0
        elif npedido == "":
           npedido=0
        elif ndata == "":
           ndata=0
        elif nipc=="":
           nipc=0
        elif ntitulo=="":
           ntitulo=0
        elif ndepositante == "":
           ndepositante=0 

        # append as listas
        rpi.append(nrpi)
        idpedido.append(npedido) 
        data.append(ndata) 
        ipc.append(nipc)
        titulo.append(ntitulo)
        depositante.append(ndepositante)
        despacho1.append(ndespacho1)
       
      i=i+1
        
  # Transformando as listas em dataframe   
  pedido=pd.DataFrame(idpedido,columns=['idpedido'])
  data=pd.DataFrame(data,columns=['data_deposito'])
  ipc=pd.DataFrame(ipc,columns=['ipc'])
  titulo=pd.DataFrame(titulo,columns=['titulo'])
  depositante=pd.DataFrame(depositante,columns=['depositante'])
  despacho=pd.DataFrame(despacho1,columns=['despacho'])
  rpi=pd.DataFrame(rpi,columns=['rpi'])

  
  # Juntando os dataframes
  frames=[pedido,data,ipc,titulo,depositante,despacho,rpi]
  pedido=pd.concat(frames, axis=1)
    
  return pedido

In [ ]:
# função para extrair as informações dos códigos 6.1, 6.20, 6.21, 6.22, 7.1
def exi_ci(list1):
  # Criando listas vazias
  rpi=[]
  despacho1=[]
  idpedido=[]
  data=[]
  depositante=[]

  i=0


  # Extração propriamente dita
  for j in list1[:]: 
      nrpi=0
      nrpi=list1[0]
    
      if list1[i][5:] == '6.1' or list1[i][5:]  == '7.1':
        
        ndespacho1=0
        npedido =0
        ndata=0
        ndepositante=0

        ndespacho1=list1[i][5:]
        npedido =list1[i+1][5:]
        ndata=list1[i+2][5:]
   
        # Este loop dentro do despacho é para verificar se o valor correto está sendo extraído
        # Caso não encontre, o valor entrará na lista como definido acima 
      
        k=i+1  
        while list1[k][1:3]!= 'Cd':
           if list1[k][1:3]=='21':
             npedido =list1[k][5:]
           elif list1[k][1:3]=='22':  
             ndata=list1[k][5:]
           elif list1[k][1:3]=='71':
             ndepositante=list1[k][5:]  
           k=k+1
        
        # Caso a variável não seja encontrada ou seja vazia "", garantiremos que seja preenchida com 0          
        if nrpi=="":
           nrpi=0
        elif ndespacho1=="":
           ndespacho1=0
        elif npedido == "":
           npedido=0
        elif ndata == "":
           ndata=0
        elif ndepositante == "":
           ndepositante=0  
                  
        # append as listas
        idpedido.append(npedido) 
        data.append(ndata) 
        despacho1.append(ndespacho1)
        depositante.append(ndepositante)
        rpi.append(nrpi)

      elif list1[i][5:]  == '6.20' or list1[i][5:] == '6.21' or list1[i][5:]  == '6.22':
        
        ndespacho1=0
        npedido =0
        ndata=0
        ndepositante=0

        ndespacho1=list1[i][5:]
        npedido =list1[i+1][5:]
        ndata=list1[i+2][5:]
      
        # Este loop dentro do despacho é para verificar se o valor correto está sendo extraído
        # Caso não encontre, o valor entrará na lista como definido acima
        
        k=i+1  
        while list1[k][1:3]!= 'Cd':
           if list1[k][1:3]=='21':
             npedido =list1[k][5:]
           elif list1[k][1:3]=='22':  
             ndata=list1[k][5:]
           elif list1[k][1:3]=='71':
             ndepositante=list1[k][5:]  
           k=k+1

        # Caso a variável não seja encontrada ou seja vazia "", garantiremos que seja preenchida com 0         
        if nrpi=="":
           nrpi=0
        elif ndespacho1=="":
           ndespacho1=0
        elif npedido == "":
           npedido=0
        elif ndata == "":
           ndata=0
        elif ndepositante == "":
           ndepositante=0  
                    
        # append as listas
        idpedido.append(npedido) 
        data.append(ndata) 
        despacho1.append(ndespacho1)
        depositante.append(ndepositante)
        rpi.append(nrpi)  

      i=i+1
        
  # Transformando as listas em dataframe   
  pedido=pd.DataFrame(idpedido,columns=['idpedido'])
  data=pd.DataFrame(data,columns=['datadeposito'])
  despacho=pd.DataFrame(despacho1,columns=['despacho'])
  depositante=pd.DataFrame(depositante,columns=['depositante'])
  rpi=pd.DataFrame(rpi,columns=['rpi'])
   
  # Juntando os dataframes
  frames=[pedido,data,depositante,despacho,rpi]
  exigencia_ciencia=pd.concat(frames, axis=1)
    
  return exigencia_ciencia

In [ ]:
def decisao(list1):
  # Criando listas vazias
  rpi=[]
  despacho1=[]
  idpedido=[]
  data=[]
  titulo=[]
  depositante=[]
  i=0

  # Extração propriamente dita
  for j in list1[:]: 
      nrpi=0
      nrpi=list1[0]
      
      if list1[i][5:]  == '9.1' or list1[i][5:] == '9.2':
        
        ndespacho1=0
        npedido =0
        ndata=0
        ntitulo=0
        ndepositante=0

        ndespacho1=list1[i][5:]
        npedido =list1[i+1][5:]
        ndata=list1[i+2][5:]

        # Este loop dentro do despacho é para verificar se o valor correto está sendo extraído
        # Caso não encontre, o valor entrará na lista como definido acima
      
        k=i+1 
        while list1[k][1:3]!='Cd':
           if list1[k][1:3]=='21':
             npedido =list1[k][5:]
           elif list1[k][1:3]=='22':  
             ndata=list1[k][5:]
           elif list1[k][1:3]=='54':  
             ntitulo=list1[k][5:] 
           elif list1[k][1:3]=='71':
             ndepositante=list1[k][5:]  
           k=k+1

        # Caso a variável não seja encontrada ou seja vazia "", garantiremos que seja preenchida com 0         
        if nrpi=="":
           nrpi=0
        elif ndespacho1=="":
           ndespacho1=0
        elif npedido == "":
           npedido=0
        elif ndata == "":
           ndata=0
        elif ntitulo=="":
           ntitulo=0
        elif ndepositante == "":
           ndepositante=0 
                  
        # append as listas
        idpedido.append(npedido) 
        data.append(ndata) 
        titulo.append(ntitulo)
        despacho1.append(ndespacho1)
        depositante.append(ndepositante)
        rpi.append(nrpi)
        
      elif list1[i][5:]  == '11.2':

        ndespacho1=0
        npedido =0
        ndata=0
        ndepositante=0

        ndespacho1=list1[i][5:]
        npedido =list1[i+1][5:]
        ndata=list1[i+2][5:]
        ntitulo="Sem título"

        # Este loop dentro do despacho é para verificar se o valor correto está sendo extraído
        # Caso não encontre, o valor entrará na lista como definido acima
      
        k=i+1 
        while list1[k][1:3]!= 'Cd':
           if list1[k][1:3]=='21':
             npedido =list1[k][5:]
           elif list1[k][1:3]=='22':  
             ndata=list1[k][5:]
           elif list1[k][1:3]=='71':
             ndepositante=list1[k][5:]  
           k=k+1

        # Caso a variável não seja encontrada ou seja vazia "", garantiremos que seja preenchida com 0          
        if nrpi=="":
           nrpi=0
        elif ndespacho1=="":
           ndespacho1=0
        elif npedido == "":
           npedido=0
        elif ndata == "":
           ndata=0
        elif ndepositante == "":
           ndepositante=0

        # append as listas
        idpedido.append(npedido) 
        data.append(ndata) 
        titulo.append(ntitulo)
        despacho1.append(ndespacho1)
        depositante.append(ndepositante)
        rpi.append(nrpi)
        

      i=i+1
        
  # Transformando as listas em dataframe   
  pedido=pd.DataFrame(idpedido,columns=['idpedido'])
  data=pd.DataFrame(data,columns=['data_deposito'])
  titulo=pd.DataFrame(titulo,columns=['titulo'])
  despacho=pd.DataFrame(despacho1,columns=['despacho'])
  depositante=pd.DataFrame(depositante,columns=['depositante'])
  rpi=pd.DataFrame(rpi,columns=['rpi'])
   
  # Juntando os dataframes
  frames=[pedido,data,titulo,depositante,despacho,rpi]
  dec=pd.concat(frames, axis=1)
    
  return dec

Vamos verificar a quantidade de despachos 1.3, 3.1 e 3.2 nas rpi's P1527.txt, P2080.txt e P2592.txt:

In [ ]:
lista1=txtplista('P1527.txt')
lista2=txtplista('P2080.txt')
lista3=txtplista('P2592.txt')

In [ ]:
# Verificando a quantidade de despachos 1.3, 3.1 e 3.2 nas revistas escolhidas para saber o número total de linhas que o dataframe deverá ter:
print('Revista P1527: ')
print("Nº de despachos 1.3:", lista1.count('(Cd) 1.3'))
print('Nº de despachos 3.1:', lista1.count('(Cd) 3.1'))
print('Nº de despachos 3.2', lista1.count('(Cd) 3.2'))
print('Número total de linhas que o dataframe deverá ter:', lista1.count('(Cd) 1.3')+lista1.count('(Cd) 3.1')+lista1.count('(Cd) 3.2'))
print('')
print('Revista P2080: ')
print("Nº de despachos 1.3:", lista2.count('(Cd) 1.3'))
print('Nº de despachos 3.1:', lista2.count('(Cd) 3.1'))
print('Nº de despachos 3.2:', lista2.count('(Cd) 3.2'))
print('Número total de linhas que o dataframe deverá ter:', lista2.count('(Cd) 1.3')+lista2.count('(Cd) 3.1')+lista2.count('(Cd) 3.2'))
print('')
print('Revista P2592: ')
print("Nº de despachos 1.3:", lista3.count('(Cd) 1.3'))
print('Nº de despachos 3.1:', lista3.count('(Cd) 3.1'))
print('Nº de despachos 3.2:', lista3.count('(Cd) 3.2'))
print('Número total de linhas que o dataframe deverá ter:', lista3.count('(Cd) 1.3')+lista3.count('(Cd) 3.1')+lista3.count('(Cd) 3.2'))
print('')

Testando a função pedidos (extração dos códigos 1.3, 3.1 e 3.2):

In [ ]:
print(pedidos(lista1).to_string())

In [ ]:
pedidos(lista1).describe()

In [ ]:
pedidos(lista1).info()

In [ ]:
print(pedidos(lista2).to_string())

In [ ]:
pedidos(lista2).describe()

In [ ]:
pedidos(lista2).info()

In [ ]:
print(pedidos(lista3).to_string())

In [ ]:
pedidos(lista3).describe()

In [ ]:
pedidos(lista3).info()

No teste de extração da RPI 1527 com a função pedidos, falhas foram identificadas após a extração, pois na coluna de idpedido em que deveria haver somente os números dos pedidos, na linha 5 havia trecho de textos. A mesma falha foi identificada na coluna data_deposito, em que foram encontrados textos em 3 linhas, onde deveria haver somente datas. Não foram identificados erros de extração nas outras colunas.

Observou-se que o erro na extração ocorreu porque em alguns registros, os títulos ou resumos apresentam continuidade de texto nas linhas seguintes, sendo que, coincidentemente, algumas destas continuações se iniciam com caracteres semelhantes a caracteres iniciais que contém o idpedido ou a data_depósito que se iniciam com (21) e (22), respectivamente. Portanto, para mitigar o erro de extração vamos procurar padrões mais específicos relativos às informações de interesse no sentido de criar um código que possa extrair os registros com acurácia.

Desta forma, no intuito de identificar estes padrões específicos, podemos verificar como aparecem os registros entre parênteses (..) que são números ou letras.

Verificaremos a seguir com mais profundidade o formato dos cógidos de todas as revistar para identificar alguns padrões e exceções destes padrões.

In [ ]:
# Criando uma função para remover espaços no início e fim de cada linha da string
def remove_(list1):
  i=0
  ni=0
  list2=[]
  for i in list1:
    i = i.lstrip()
    ni = i.rstrip()
    list2.append(ni)
  return list2

In [ ]:
import glob

l=1
codigos=[]
rpi1=[]
rpi2=[]
for x in glob.glob("*"):
  if x.endswith(".txt") or x.endswith(".TXT"):
    lista=txtplista(x)
    lista=remove_(lista)
    rpi1.append(lista[0])
    for i in lista:
      if (i[0:1]=='(' or i[0:1]==')') and (i[1:3].isnumeric()==True or i[1:3].isalpha()==True):
        n=i[0:5]
        codigos.append(n)
        rpi2.append(lista[0])
    print('rpi:',lista[0]," arquivo:",l)
    l=l+1
print('\n') 
print("Processo de verificação Finalizado!")

In [ ]:
import os
os.remove('P2236.txt')

In [ ]:
f1=pd.DataFrame(codigos,columns=['codigos'])
f2=pd.DataFrame(rpi2,columns=['rpi'])
frames=[f2,f1]
cod_rpi=pd.concat(frames, axis=1)

In [ ]:
editado=cod_rpi[['codigos', 'rpi']].groupby('codigos').agg(lambda x: list(set(x))).reset_index()

In [ ]:
print(editado.to_string())

Da visualização acima concluímos que em todos os arquivos txt, na linha 0 consta o número da rpi e a data de publicação. Também é notório que há uma algumas variações na forma em que a informação foi registrada no arquivo txt.

Outros padrões foram identificados e vamos utilizá-los para melhorar a extração e evitar dados indesejáveis:

- Após o código do nº do pedido (21) o pedido sempre se inicia com letra maiuscula;

- Após o código da data de depósito (22) a data sempre se inicia com um número;

- Após o código da ipc (51), a classificação sempre é grafada em letra maiúscula;

- O título, após o código (54), sempre se inicia com uma letra maiúscula, ou outro símbolo que não uma letra.

Percebe-se também que pode-se extrair tudo utilizando apenas uma função, mas para essa única função funcionar, precisamos tirar todos os espaços do início e do fim de cada linha da lista criada a partir de cada arquivo txt.

Algumas variações no registro da informação (Cd):

- 538 '(cd) ' [No. 1541 de 18/07/2000, No. 1539 04/07/2000, No. 1540 11/07/2000]

- 300 '(Cd) ' [No 2474 de 05/06/2018, No 2060 de 29/06/2010, ...]

- 280 '(CD) ' [No. 1809 06/09/2005, No. 1811 20/09/2005, ...]

Assim, as linhas da lista que possuem os digitos cd, podem assumir as formas vistas acima. Para contornar esta não padronização nos registros, vamos criar uma função para transformar todos os dígitos alphanuméricos entre () que estão no início de cada linha da lista para caixa alta:


In [ ]:
def c_alta(list1):
  i=0
  ni=0
  list2=[]
  for i in list1:
    if i[0:1]=='(' and i[1:3].isalpha()==True:
      ni=i.upper()
    else:
      ni=i  
    list2.append(ni)
  return list2

In [ ]:
lista1=txtplista('P2236.txt')
lista1

In [ ]:
c_alta(lista1)

Modificando o código de extração com base nas observações anteriores. 
Teremos que extrair os dados dos despachos 2.4 ou 23.1.1 também, pois são pedidos divididos, juntamente com o número do pedido original, para posterior extração dos dados bibliográficos dos pedidos originais, que são os mesmos, podendo o título e/ou classificação serem os mesmos ou serem distintos, não haverá como obter estes dados. Outro dado, não menos importante, que temos que levar em consideração é o fato de que os números dos pedidos podem ser alterados por conta de mudança da natureza (patente de invenção para patente de modelo de utilidade ou vice-versa) ou por ter sido publicada com numeração indevida. Para extrair estes dados, a função de extração retornará um dataframe com o número do pedido e a nova numeração, para posterior ajuste dos dataframes principais.

Os despachos que publicam a nova numeração do pedido são 15.10, 15.12 e 51.

Vamos mudar um pouco a estratégia de estração, ao invés de termos três funções, vamos criar apenas uma, que retornará 3 dataframes:

In [ ]:
import pandas as pd

In [ ]:
# função para extrair as informações dasRPI’s e transformar estar informações em 3 dataframes  
def pedidos(list1):
  
  # Criando listas vazias
  data_rpi=[]
  despacho1=[]
  idpedido =[] 
  data=[]
  original=[]
  ipc=[]
  titulo=[]
  depositante=[]
  
  data_rpi2=[]
  despacho2=[]
  idpedido2=[]
  depositante2=[]
  data2=[]

  data_rpi3=[]
  despacho3=[]
  idpedido3=[]
  novo_numero=[]

  ndata_rpi=''
  # vamos trabalhar apenas com a data de publicação da rpi
  # a linha abaixo retorna apenas os últimos caracteres da linha 0 da lista
  ndata_rpi=list1[0][list1[0].rindex(' ')+1:] 

  ndata_rpi2=''
  ndata_rpi2=ndata_rpi 
  
  ndata_rpi3=''
  ndata_rpi3=ndata_rpi 
 
  i=0  # linha da lista que será extraída a informação
  be=0
  ce=0
  de=0

  # Extração propriamente dita
  for j in list1:
      b=['(CD) 1.3','(CD) 3.1','(CD) 3.2','(CD) 2.4','(CD) 23.1.1']
      if j in b:
        if list1[i+1][5:7]=='PI' or list1[i+1][5:7]=='MU' or list1[i+1][5:6]=='C' or list1[i+1][5:7]=='BR':
          be+=1 # contagem dos pedidos com despachos contidos em b
          ndespacho1=''
          npedido =''
          ndata=''
          noriginal=''
          nipc=''
          ntitulo=''
          ndepositante=''

          ndespacho1=list1[i][5:]
                   
          # Este loop dentro do despacho é para verificar se o valor correto está sendo extraído
          k=i+1
          while list1[k][0:5] != '(CD) ':
             if list1[k][0:5]=='(21) ' and list1[k][5:6].isupper()==True:
               npedido =list1[k][5:]
             elif list1[k][0:5]=='(22) ' and list1[k][5:6].isnumeric()==True:
               ndata=list1[k][5:]
             elif list1[k][0:5]=='(62) ' and list1[k][5:6].isupper()==True:
               noriginal=list1[k][5:]
             elif list1[k][0:5]=='(51) ' and list1[k][5:6].isupper()==True:
               nipc=list1[k][5:]
             elif list1[k][0:5]=='(54) ' and (list1[k][5:6].isalpha()==False or list1[k][5:6].isupper()==True):
               ntitulo=list1[k][5:]
             elif list1[k][0:5]=='(71) ': 
               ndepositante=list1[k][5:]
             k+=1

                  
          # append as listas
          data_rpi.append(ndata_rpi)
          idpedido.append(npedido) 
          data.append(ndata)
          original.append(noriginal) 
          ipc.append(nipc)
          titulo.append(ntitulo)
          depositante.append(ndepositante)
          despacho1.append(ndespacho1)         

      c=['(CD) 6.1','(CD) 6.20','(CD) 6.21','(CD) 6.22','(CD) 7.1','(CD) 8.6','(CD) 8.11','(CD) 8.12','(CD) 9.1','(CD) 9.2','(CD) 11.1.1','(CD) 11.2', '(CD) 11.4', '(CD) 11.5','(CD) 11.6','(CD) 11.11','(CD) 11.18','(CD) 11.30','(CD) 11.31']
      if j in c:
        if list1[i+1][5:7]=='PI' or list1[i+1][5:7]=='MU' or list1[i+1][5:6]=='C' or list1[i+1][5:7]=='BR':
          ce+=1 # contagem dos pedidos com despachos contidos em c
          ndespacho2=''
          npedido2 =''
          ndata2=''
          ndepositante2=''
          ndespacho2=list1[i][5:]
          
          # Este loop dentro do despacho é para verificar se o valor correto está sendo extraído
          k=i+1
          while list1[k][0:5] != '(CD) ':
             if list1[k][0:5]=='(21) ' and list1[k][5:6].isupper()==True:
               npedido2 =list1[k][5:]
             elif list1[k][0:5]=='(22) ' and list1[k][5:6].isnumeric()==True:
               ndata2=list1[k][5:]
             elif list1[k][0:5]=='(71) ': 
               ndepositante2=list1[k][5:]
             k+=1

          # append as listas
          data_rpi2.append(ndata_rpi2)
          idpedido2.append(npedido2)
          depositante2.append(ndepositante2)
          data2.append(ndata2) 
          despacho2.append(ndespacho2)

      d=['(CD) 15.10','(CD) 15.12','(CD) 51']
      if j in d:
        if list1[i+1][5:7]=='PI' or list1[i+1][5:7]=='MU' or list1[i+1][5:6]=='C' or list1[i+1][5:7]=='BR':
          de+=1 # contagem dos pedidos com despachos contidos em d
          ndespacho3=''
          npedido3 =''
          nnovo_numero=''
          
          # Este loop dentro do despacho é para verificar se o valor correto está sendo extraído
          k=i+1
          while list1[k][0:5] != '(CD) ':
             if list1[k][0:5]=='(21) ' and list1[k][5:6].isupper()==True:
               npedido3 =list1[k][5:]
             elif list1[k][0:5]=='(CO) ':
               nnovo_numero=list1[k][5:]
             k+=1
          # append as listas
          data_rpi3.append(ndata_rpi3)
          idpedido3.append(npedido3)
          novo_numero.append(nnovo_numero)

      i+=1
        
  # Transformando as listas em dataframe   
  pedido=pd.DataFrame(idpedido,columns=['idpedido'])
  data=pd.DataFrame(data,columns=['data_deposito'])
  original=pd.DataFrame(original,columns=['original'])
  ipc=pd.DataFrame(ipc,columns=['ipc'])
  titulo=pd.DataFrame(titulo,columns=['titulo'])
  depositante=pd.DataFrame(depositante,columns=['depositante'])
  despacho=pd.DataFrame(despacho1,columns=['despacho'])
  data_rpi=pd.DataFrame(data_rpi,columns=['data_rpi'])

  pedido2=pd.DataFrame(idpedido2,columns=['idpedido'])
  depositante2=pd.DataFrame(depositante2,columns=['depositante'])
  data2=pd.DataFrame(data2,columns=['data_deposito'])
  despacho2=pd.DataFrame(despacho2,columns=['despacho'])
  data_rpi2=pd.DataFrame(data_rpi2,columns=['data_rpi'])

  pedido3=pd.DataFrame(idpedido3,columns=['idpedido'])
  novo_numero=pd.DataFrame(novo_numero,columns=['novo_numero'])
  data_rpi3=pd.DataFrame(data_rpi3,columns=['data_rpi'])
  
  # Juntando os dataframes
  frames=[pedido,data,original,ipc,titulo,depositante,despacho,data_rpi]
  pedido=pd.concat(frames, axis=1)

  frames2=[pedido2,data2,depositante2,despacho2,data_rpi2]
  outros=pd.concat(frames2, axis=1)

  frames3=[pedido3,novo_numero,data_rpi3]
  nova_num=pd.concat(frames3, axis=1)
    
  return pedido,outros,nova_num,be,ce,de

Testando novamente a função pedidos:

In [ ]:
lista1=txtplista('P1527.txt')
lista1=remove_(lista1)
lista1=c_alta(lista1)
lista2=txtplista('P2080.txt')
lista2=remove_(lista2)
lista2=c_alta(lista2)
lista3=txtplista('P2592.txt')
lista3=remove_(lista3)
lista3=c_alta(lista3)

In [ ]:
# Verificando a quantidade de despachos 1.3, 3.1 e 3.2 nas revistas escolhidas para saber o número total de linhas que o dataframe deverá ter:
print('Revista P1527: ')
print("Nº de despachos 1.3:", lista1.count('(CD) 1.3'))
print('Nº de despachos 3.1:', lista1.count('(CD) 3.1'))
print('Nº de despachos 3.2', lista1.count('(CD) 3.2'))
print('Nº de despachos 2.4', lista1.count('(CD) 2.4'))
print('Nº de despachos 23.1.1', lista1.count('(CD) 23.1.1'))
print('Número total de linhas que o dataframe deverá ter:', lista1.count('(CD) 1.3')+lista1.count('(CD) 3.1')+lista1.count('(CD) 3.2')+lista1.count('(CD) 2.4')+lista1.count('(CD) 23.1.1'))
print('')
print('Revista P2080: ')
print("Nº de despachos 1.3:", lista2.count('(CD) 1.3'))
print('Nº de despachos 3.1:', lista2.count('(CD) 3.1'))
print('Nº de despachos 3.2:', lista2.count('(CD) 3.2'))
print('Nº de despachos 2.4', lista2.count('(CD) 2.4'))
print('Nº de despachos 23.1.1', lista2.count('(CD) 23.1.1'))
print('Número total de linhas que o dataframe deverá ter:', lista2.count('(CD) 1.3')+lista2.count('(CD) 3.1')+lista2.count('(CD) 3.2')+lista2.count('(CD) 2.4')+lista2.count('(CD) 23.1.1'))
print('')
print('Revista P2592: ')
print("Nº de despachos 1.3:", lista3.count('(CD) 1.3'))
print('Nº de despachos 3.1:', lista3.count('(CD) 3.1'))
print('Nº de despachos 3.2:', lista3.count('(CD) 3.2'))
print('Nº de despachos 2.4', lista3.count('(CD) 2.4'))
print('Nº de despachos 23.1.1', lista3.count('(CD) 23.1.1'))
print('Número total de linhas que o dataframe deverá ter:', lista3.count('(CD) 1.3')+lista3.count('(CD) 3.1')+lista3.count('(CD) 3.2')+lista3.count('(CD) 2.4')+lista3.count('(CD) 23.1.1'))
print('')

In [ ]:
import pandas as pd
total,outros,nova_num,a,b,c = pedidos(lista1)

In [ ]:
print(total.to_string())

In [ ]:
print(outros.to_string())

In [ ]:
print(nova_num.to_string())

In [ ]:
print(a,'',b,'',c)

In [ ]:
total, outros,nova_num,a,b,c=pedidos(lista2)

In [ ]:
print(total.to_string())

In [ ]:
print(outros.to_string())

In [ ]:
print(nova_num.to_string())

In [ ]:
print(a,'',b,'',c)

In [ ]:
total, outros,nova_num,a,b,c=pedidos(lista3)

In [ ]:
print(total.to_string())

In [ ]:
print(outros.to_string())

In [ ]:
print(nova_num.to_string())

Este último teste confirma que a função pedidos realiza a extração das informações de forma adequada.

Agora, após as verificações mostrarem que a função extraiu as informações de forma adequada, vamos extrair as informações de todos os arquivos txt que temos na pasta. 


Aplicando a função de extração arquivo por arquivo, guardando cada dataframe gerado em uma lista para depois concatenar todos os dataframes e, por fim, salvar os dataframes em um formato csv:

In [ ]:
import pandas as pd
import glob
from datetime import datetime

ped_pub=[]
outros=[]
nova_num=[]
xl=[]
al=[]
a1=0
b1=0
c1=0
start_time = datetime.now()
l=1
for x in glob.glob("*"):
  if x.endswith(".txt") or x.endswith(".TXT"):
    lista1=[]
    # Abrindo o arquivo e transformando-o em uma lista
    lista1=txtplista(x)
    # Tirando os espaços do início e do fim de cada linha da lista
    lista1=remove_(lista1)
    # Passando todos os registros (cd) ou (co) para caixa alta
    lista1=c_alta(lista1)
    # Extraindo as informações e salvando-as em 2 dataframes
    df1,df2,df3,a,b,c=pedidos(lista1)
    # append os df
    ped_pub.append(df1)
    outros.append(df2)
    nova_num.append(df3)
    xl.append(x)
    al.append(a)

    a1=a1+a
    b1=b1+b
    c1=c1+c

    print('Extraindo os dados do arquivo',x,'... ',' Nº de 1.3, 3.1, 3.2, 2.4 e 23.1.1:',a,' Arquivo nº:',l)
    l+=1
end_time = datetime.now()

print('\n')
print('Total de linhas do dataframe com publicações 1.3, 3.1, 3.2, 2.4 e 23.1.1: ', a1)

print('\n')
print('Total de linhas do dataframe com outras publicações: ', b1)

print('\n')
print('Total de linhas do dataframe com pedidos renumerados: ', c1)

print('\n')
print('Tempo de extração das informações: {}'.format(end_time - start_time))

# Concatenando os df
print('\n')
print('Concatenando os dataframes...')
xdf = pd.DataFrame({'rpi':xl})
adf = pd.DataFrame({'qtd_pub':al})
fr=[xdf,adf]
xdfadf=pd.concat(fr, axis=1)
dep_pub_total=pd.concat(ped_pub,ignore_index=True)
outros_total=pd.concat(outros,ignore_index=True)
nova_num_total=pd.concat(nova_num,ignore_index=True)

# Salvando os dataframes no formato csv
print('\n')
print('Salvando os dataframes no formato csv...')
xdfadf.to_csv("xdfadf.csv", index=False)
dep_pub_total.to_csv("dep_pub_total.csv", index=False)
outros_total.to_csv("outros_total.csv", index=False) 
nova_num_total.to_csv("nova_num_total.csv", index=False) 

print('\n')
print('Documentos salvos na pasta!')
print('\n') 
print("Processo de extração Finalizado!")

Os dataframes estão salvos na pasta! Vamos visualizar os dataframes gerados e fazer algumas verificações:

In [ ]:
!ls

In [ ]:
import pandas as pd
xdfadf=pd.read_csv('xdfadf.csv')
xdfadf.sort_values(by=['qtd_pub']).head(60)

Após verificar as revistas que constam 0 no número de pedidos publicados com despachos 1.3, 3.1, 3.2 e 3.4, constatamos que as RPI's 2119	e 2118	possuem sim estes despachos publicados, não nos arquivos txt, mas somente nos arquivos pdf publicados.

Foi verificado que as outras RPI's que constam 0 nas publicações, realmente não possuem os citados despachos.

Vamos readaptar a função para extração dos dados somente para estas duas revistas. Dado que ao extrair os textos dos arquivos pdf o arquivo contendo o texto será diferente do arquivo txt extraído do website do INPI.

Para extrair os dados de interesse vamos utilizar a biblioteca Apache Tika para a extração de texto de arquivos pdf, pois foi o método que demonstrou melhor extração após um teste rápido com as bibliotecas PyPDF2, Tabula, PDFMiner e o próprio Apache Tika.

Vamos ao código:

In [ ]:
# Instalando a biblioteca Apache Tika
!pip install tika

In [ ]:
from tika import parser

In [ ]:
# Extraindo o texto do pdf...
dados_pdf1 = parser.from_file('PATENTES2118.pdf')
arquivo_txt1 = dados_pdf1['content']

dados_pdf2 = parser.from_file('PATENTES2119.pdf')
arquivo_txt2 = dados_pdf2['content']

In [ ]:
# Formatando as linhas 
textlines1 = [textline1 for textline1 in arquivo_txt1.splitlines() if textline1 != '' and textline1 != ' ']
textlines2 = [textline2 for textline2 in arquivo_txt2.splitlines() if textline2 != '' and textline2 != ' ']

In [ ]:
# Tirando os espaços do início e do fim de cada linha da lista
l2118=remove_(textlines1)
l2119=remove_(textlines2)

In [ ]:
l2118

In [ ]:
l2119

Vamos mudar a estratégia de extração, fazendo uma adaptação no algoritmo de extração:

In [ ]:
# função para extrair as informações das RPI’s 2118 e 2119  
def restante(list1,data_rpi1):
  
  # Criando listas vazias
  data_rpi=[]
  despacho1=[]
  idpedido =[] 
  data=[]
  original=[]
  ipc=[]
  titulo=[]
  depositante=[]
  ndata_rpi=''
 
  i=0  # linha da lista que será extraída a informação
  be=0
  k=0
  # Extração propriamente dita
  for j in list1:
        if list1[i][0:4]=='(21)' and k<(len(list1)-1) and (list1[i][list1[i].rindex(' ')+1:]=='1.3' or list1[i][list1[i].rindex(' ')+1:]=='3.1' or list1[i][list1[i].rindex(' ')+1:]=='3.2' or list1[i][list1[i].rindex(' ')+1:]=='2.4'):
          be+=1 # contagem dos pedidos com despachos contidos em b
          ndespacho1=''
          npedido =''
          ndata=''
          noriginal=''
          nipc=''
          ntitulo=''
          ndepositante=''

          ndata_rpi=data_rpi1
          npedido =j[5:17]
          ndata=j[26:37] 
          ndespacho1=list1[i][list1[i].rindex(' ')+1:]
        
          # Este loop dentro do despacho é para verificar se o valor correto está sendo extraído
          k=i+1
          while list1[k][0:4] != '(21)':
             if list1[k][0:4]=='(62)':
               noriginal=list1[k][5:]
             elif list1[k][0:4]=='(51)':
               nipc=list1[k][5:]
             elif list1[k][0:4]=='(54)':
               ntitulo=list1[k][5:]
             elif list1[k][0:4]=='(71)': 
               ndepositante=list1[k][5:]
             k+=1
             
          # append as listas
          data_rpi.append(ndata_rpi)
          idpedido.append(npedido) 
          data.append(ndata)
          original.append(noriginal) 
          ipc.append(nipc)
          titulo.append(ntitulo)
          depositante.append(ndepositante)
          despacho1.append(ndespacho1)         


        i+=1
        
  # Transformando as listas em dataframe   
  pedido=pd.DataFrame(idpedido,columns=['idpedido'])
  data=pd.DataFrame(data,columns=['data_deposito'])
  original=pd.DataFrame(original,columns=['original'])
  ipc=pd.DataFrame(ipc,columns=['ipc'])
  titulo=pd.DataFrame(titulo,columns=['titulo'])
  depositante=pd.DataFrame(depositante,columns=['depositante'])
  despacho=pd.DataFrame(despacho1,columns=['despacho'])
  data_rpi=pd.DataFrame(data_rpi,columns=['data_rpi'])

 
  # Juntando os dataframes
  frames=[pedido,data,original,ipc,titulo,depositante,despacho,data_rpi]
  pedido=pd.concat(frames, axis=1)

    
  return pedido,be

In [ ]:
# Extraindo as informações e salvando-as em 2 dataframes
data_rpi1='09/08/2011'
data_rpi2='16/08/2011'
df2118,a=restante(l2118,data_rpi1)
df2119,a=restante(l2119,data_rpi2)

In [ ]:
df2118.sort_values(by=['ipc']).head(60)

In [ ]:
df2118.sort_values(by=['ipc']).tail(60)

In [ ]:
df2119.sort_values(by=['ipc']).head(60)

In [ ]:
df2119.sort_values(by=['ipc']).tail(60)

A extração dos despachos 1.3, 3.1, 3.2 e 2.4 foram extraídas com precisão dos arquivos pdf das RPI's 2118 e 2119.
Vamos juntar os dois dataframes: 

In [ ]:
fr=[df2118,df2119]
dfrestante=pd.concat(fr)
dfrestante.to_csv("dfrestante.csv", index=False)
dfrestante

In [ ]:
ped_pub=pd.read_csv('dep_pub_total.csv')
ped_pub

In [ ]:
outros_total=pd.read_csv('outros_total.csv')
outros_total

In [ ]:
nova_num_total=pd.read_csv('nova_num_total.csv')
nova_num_total

In [ ]:
ped_pub.sort_values(by=['idpedido']).head(60)

In [ ]:
ped_pub.sort_values(by=['idpedido']).tail(60)

In [ ]:
ped_pub.sort_values(by=['data_deposito']).head(60)

In [ ]:
ped_pub.sort_values(by=['data_deposito']).tail(60)

In [ ]:
ped_pub.sort_values(by=['ipc']).head(60)

In [ ]:
ped_pub.sort_values(by=['ipc']).tail(60)

In [ ]:
ped_pub.sort_values(by=['titulo']).head(60)

In [ ]:
ped_pub.sort_values(by=['titulo']).tail(60)

In [ ]:
ped_pub.sort_values(by=['depositante']).head(60)

In [ ]:
ped_pub.sort_values(by=['depositante']).tail(60)

In [ ]:
ped_pub['despacho'] = ped_pub['despacho'].astype(str)

In [ ]:
ped_pub.sort_values(by=['despacho']).head(60)

In [ ]:
ped_pub.sort_values(by=['despacho']).tail(60)

In [ ]:
ped_pub.sort_values(by=['data_rpi']).head(60)

In [ ]:
ped_pub.sort_values(by=['data_rpi']).tail(60)

In [ ]:
ped_pub.info()

In [ ]:
outros_total.sort_values(by=['idpedido']).head(60)

In [ ]:
outros_total.sort_values(by=['idpedido']).tail(60)

In [ ]:
outros_total.sort_values(by=['despacho']).head(60)

In [ ]:
outros_total.sort_values(by=['despacho']).tail(60)

In [ ]:
outros_total.sort_values(by=['data_rpi']).head(60)

In [ ]:
outros_total.sort_values(by=['data_rpi']).tail(60)

In [ ]:
outros_total.info()

In [ ]:
nova_num_total.sort_values(by=['idpedido']).head(60)

In [ ]:
nova_num_total.sort_values(by=['idpedido']).tail(60)

In [ ]:
nova_num_total.sort_values(by=['novo_numero']).head(60)

In [ ]:
nova_num_total.sort_values(by=['novo_numero']).tail(60)

In [ ]:
nova_num_total.sort_values(by=['data_rpi']).head(60)

In [ ]:
nova_num_total.sort_values(by=['data_rpi']).tail(60)

In [ ]:
nova_num_total.info()

Pelas visualizações acima, verificamos que os dados foram extraídos de forma adequada. Vamos explorar mais os dtaframes nas etapas seguintes.

Finalizamos a extração das informações, vamos à transformação dos dados!

# **2.   Transformação**




Pelas verificações acima, conclui-se que a etapa de transformação e limpeza dos dados faz-se necessária, devido à discrepância dos registros publicados nas RPI’s. Portanto, com base nas verificações feitas, algumas padronizações serão realizadas na etapa de transformação, tais como:

1)	Tratar o dataframe nova_num_total no Excel, de forma deixar somente duas colunas (o número antigo e a nova numeração). Alguns ajustes terão que ser feitos manualmente, tendo em vista o fato de que a coluna novo_numero contém os comentários, os quais possuem o novo número do pedido. Algumas linhas foram excluídas, pois não tratavam de mudança de numeração ou era a renumeração de pedidos pipelines, que não serão estudados no presente trabalho. Este tratamento será feito no excel, dado que não há uma padronização nas alterações dos números dos pedidos, sendo que muitos passos terão que ser feitos manualmente;

2)	Criar uma nova coluna, que será a coluna chave, a qual será denominada de chave_idpedido. Nesta coluna serão removidas todas as strings, inclusive, após o traço “-“. A coluna idpedido será mantida;

3)	Padronizar as colunas da data de depósito e data da RPI nos dois dataframes, pois estas datas possuíam diversos formatos nas RPI’s: dd/mm/yyyy, dd/mm/yy ou ddmmyy. O formato padrão escolhido foi yyyy-mm-dd;

4)	Criar uma coluna contendo somente a primeira ipc do pedido no dataframe “ped_pub” (sem o ano de atualização da IPC, informação que consta nas revistas mais recentes, entre parênteses) para simplificação de análise, pois há pedidos que chegam a possuir mais de cinco classificações IPC;

5)	Excluir os caracteres especiais (#$&? etc.) dos títulos e passar todos os caracteres dos títulos para caixa alta no dataframe “ped_pub”;

6)	Criar uma coluna contendo somente o primeiro depositante e passando os caracteres para caixa alta nos dois dataframes;

7)	Criar mais três colunas subdividindo as classificações IPC: ipc_1 contém a seção da IPC (primeira letra da IPC), ipc_2 contém a subseção da ipc (três primeiros caracteres da IPC) e ipc_3 contém a classe da IPC (quatro primeiros caracteres da IPC) no dataframe “ped_pub”. Em suma, quanto maior a subdivisão, maior o detalhamento técnico informado pela IPC;

8)	Criar uma coluna informando o país de origem do depositante e outra contendo o estado, caso o país depositante seja o Brasil (BR);

9)	Verificar as repetições na coluna “idpedido” do dataframe “ped_pub”, e excluir as repetições mantendo-se o registro que contém a publicação na RPI mais recente;

10)	Criar uma nova coluna contendo a descrição da área e campo tecnológicos de acordo com a classificação IPC, segundo a OMPI – Organização Mundial da Propriedade Intelectual, utilizando o método merge;

11)	Criar uma nova coluna informando se o depositante é Residente (Brasil) ou Não-Residente (Internacional);

12)	Por fim, juntar os dois dataframes utilizando o método merge. 



In [ ]:
!ls

In [ ]:
import pandas as pd

In [ ]:
ped_pub=pd.read_csv('dep_pub_total.csv')
dfrestante=pd.read_csv('dfrestante.csv')
outros=pd.read_csv("outros_total.csv")
nova_num_total_ed=pd.read_csv("nova_num_total_ed.csv",encoding='iso-8859-1',delimiter=';')

In [ ]:
ped_pub['despacho'] = ped_pub['despacho'].astype(str)

In [ ]:
dfrestante['despacho']=dfrestante['despacho'].astype(str)

In [ ]:
# Verificando o tipo de cada coluna n
print(ped_pub.info())
print(outros.info())
print(nova_num_total_ed.info())
print(dfrestante.info())

Criando uma nova coluna chave_idpedido nos dataframes:

In [ ]:
ped_pub

In [ ]:
outros

In [ ]:
nova_num_total_ed

O dataframe nova_num_total_ed foi previamente trado no excel, vamos apenas renomear as colunas:

In [ ]:
nova_num_total_ed.columns = ['chave_idpedido', 'nova_chave_idpedido'] 
nova_num_total_ed

In [ ]:
ped_pub['chave_idpedido'] = ped_pub['idpedido']
outros['chave_idpedido'] = outros['idpedido']
dfrestante['chave_idpedido']=dfrestante['idpedido']

In [ ]:
# Tirando os espaços no início e no fim de cada string da coluna chave_idpedido
ped_pub['chave_idpedido'] = ped_pub['chave_idpedido'].str.lstrip()
ped_pub['chave_idpedido'] = ped_pub['chave_idpedido'].str.rstrip()

outros['chave_idpedido'] = outros['chave_idpedido'].str.lstrip()
outros['chave_idpedido'] = outros['chave_idpedido'].str.rstrip()

dfrestante['chave_idpedido']=dfrestante['chave_idpedido'].str.lstrip()
dfrestante['chave_idpedido']=dfrestante['chave_idpedido'].str.rstrip()

In [ ]:
# Excluindo os kind codes da coluna chave_idpedido (consultar o site https://www.wipo.int/export/sites/www/standards/en/pdf/07-03-02.pdf para maiores informações)
x=['A2','A8','B1','B8','C8','E2','E8','F1','F8','G8','U2','U8','Y1','Y8','Z8']
for i in x:
  ped_pub['chave_idpedido'] = ped_pub['chave_idpedido'].map(lambda x: x.replace(i,''))
  outros['chave_idpedido'] = outros['chave_idpedido'].map(lambda x: x.replace(i,''))
  nova_num_total_ed['chave_idpedido'] = nova_num_total_ed['chave_idpedido'].map(lambda x: x.replace(i,''))
  nova_num_total_ed['nova_chave_idpedido'] = nova_num_total_ed['nova_chave_idpedido'].map(lambda x: x.replace(i,''))

In [ ]:
# Tirando os espaços no início e no fim de cada string da coluna chave_idpedido novamente
ped_pub['chave_idpedido'] = ped_pub['chave_idpedido'].str.lstrip()
ped_pub['chave_idpedido'] = ped_pub['chave_idpedido'].str.rstrip()

outros['chave_idpedido'] = outros['chave_idpedido'].str.lstrip()
outros['chave_idpedido'] = outros['chave_idpedido'].str.rstrip()


In [ ]:
# Cada pedido de patente possui, antes do kind code, possuir um ifem seguindo de um número, que pode ser de 0 a 9. Vamos excluir estes números e o ifem também da chave_idpedido
y=['-0','-1','-2','-3','-4','-5','-6','-7','-8','-9']
for j in y:
  ped_pub['chave_idpedido'] = ped_pub['chave_idpedido'].map(lambda x: x.replace(j,''))
  outros['chave_idpedido'] = outros['chave_idpedido'].map(lambda x: x.replace(j,''))
  dfrestante['chave_idpedido']=dfrestante['chave_idpedido'].map(lambda x: x.replace(j,''))

In [ ]:
# Removendo os espaços em todas as strings da coluna chave_idpedido
ped_pub['chave_idpedido'] = ped_pub['chave_idpedido'].str.replace(' ', '')
outros['chave_idpedido'] = outros['chave_idpedido'].str.replace(' ', '')
nova_num_total_ed['chave_idpedido'] = nova_num_total_ed['chave_idpedido'].str.replace(' ', '')
nova_num_total_ed['nova_chave_idpedido'] = nova_num_total_ed['nova_chave_idpedido'].str.replace(' ', '')
dfrestante['chave_idpedido']=dfrestante['chave_idpedido'].str.replace(' ', '')

In [ ]:
# Renomeando as colunas
ped_pub=ped_pub[['chave_idpedido','idpedido','data_deposito', 'original','ipc','titulo','depositante','despacho','data_rpi']]
outros=outros[['chave_idpedido','idpedido','data_deposito','depositante','despacho','data_rpi']]
dfrestante=dfrestante[['chave_idpedido','idpedido','data_deposito', 'original','ipc','titulo','depositante','despacho','data_rpi']]

In [ ]:
ped_pub

In [ ]:
outros

In [ ]:
nova_num_total_ed

In [ ]:
dfrestante

Adicionando uma nova coluna ao dataframe contendo apenas a primeira classificação:

In [ ]:
# Função para pegar apenas a primeira classificação da coluna ipc
def tira(test_str):
    saida = ''
    v1 = 0
    v2 = 0
    for i in str(test_str):
        if i == ',':
            v1 += 1
        elif i == '(':
            v2 += 1
        elif i == ',' and v1 > 0:
            v1 -= 1
        elif i == ')'and v2 > 0:
            v2 -= 1
        elif v1 == 0 and v2 == 0:
            saida += i
    return saida

In [ ]:
# Extraindo somente a primeira classificação da coluna ipc, para criar a coluna ipc1
ped_pub['ipc1'] = ped_pub['ipc'].apply(lambda x: tira(x))
dfrestante['ipc1']=dfrestante['ipc'].apply(lambda x: tira(x))

In [ ]:
ped_pub

In [ ]:
dfrestante

Removendo os caracteres especiais da coluna titulo e aplicando a função upper em toda a coluna:

In [ ]:
import re
ped_pub['titulo1'] = ped_pub['titulo'].map(lambda x: re.sub(r'\W+', ' ', str(x)))
ped_pub['titulo1']=ped_pub['titulo1'].str.upper()
dfrestante['titulo1'] = dfrestante['titulo'].map(lambda x: re.sub(r'\W+', ' ', str(x)))
dfrestante['titulo1']=dfrestante['titulo1'].str.upper()

In [ ]:
ped_pub

In [ ]:
dfrestante

Modificando a função que foi aplicada às classificações, vamos criar uma coluna somente com o nome do 1º depositante e tudo em maiúsculo:

In [ ]:
# Função para pegar apenas o primeiro depositante com o país de origem
def tira1(test_str):
    saida = ''
    v1 = 0
    for i in str(test_str):
        if i == ';':
            v1 += 1
        elif i == ';'  and v1>0:
            v1 -= 1
        elif v1 == 0:
            saida += i
    return saida

In [ ]:
# Extraindo somente o primeiro depositante com o país de origem
ped_pub['atalho'] = ped_pub['depositante'].apply(lambda x: tira1(x))
ped_pub['atalho']=ped_pub['atalho'].str.upper()

outros['atalho'] = outros['depositante'].apply(lambda x: tira1(x))
outros['atalho']=outros['atalho'].str.upper()

dfrestante['atalho'] = dfrestante['depositante'].apply(lambda x: tira1(x))
dfrestante['atalho']=dfrestante['atalho'].str.upper()

In [ ]:
ped_pub

In [ ]:
outros

In [ ]:
dfrestante

Vamos pegar somente o depositante da coluna atalho e criar uma coluna depositante1 para armazenar estes dados:

In [ ]:
# Função para pegar apenas o primeiro depositante sem o país de origem
def tira2(test_str):
    saida = ''
    v2 = 0
    for i in str(test_str):
        if i == '(':
            v2 += 1
        elif i == ')'and v2 > 0:
            v2 -= 1
        elif v2 == 0:
            saida += i
    return saida

In [ ]:
# Extraindo somente o primeiro depositante sem o país de origem
ped_pub['depositante1'] = ped_pub['atalho'].apply(lambda x: tira2(x))
outros['depositante1'] = outros['atalho'].apply(lambda x: tira2(x))
dfrestante['depositante1'] = dfrestante['atalho'].apply(lambda x: tira2(x))

In [ ]:
ped_pub

In [ ]:
outros

In [ ]:
dfrestante

Na coluna atalho podem exister alguns registros contendo outras strings entre parenteses, o que causa erro na extração do país do depositante. Vamos pegar então somente as 7 últimas posições da coluna atalho, para evitar que strings indesejadas sejam extraídas:

In [ ]:
ped_pub['atalho2'] = ped_pub['atalho'].apply(lambda x: x[-7:])
outros['atalho2'] = outros['atalho'].apply(lambda x: x[-7:])
dfrestante['atalho2'] = dfrestante['atalho'].apply(lambda x: x[-7:])

In [ ]:
ped_pub

In [ ]:
outros

In [ ]:
dfrestante

In [ ]:
# Função para pegar apenas o país do depositante
def tira3(test_str):
    saida = ''
    v2 = 0
    for i in str(test_str):
        if i == '(':
            v2 += 1
        elif i == ')'and v2 > 0:
            v2 -= 1
        elif v2 != 0:
            saida += i
    return saida

In [ ]:
ped_pub['atalho2'] = ped_pub['atalho2'].apply(lambda x: tira3(x))
outros['atalho2'] = outros['atalho2'].apply(lambda x: tira3(x))
dfrestante['atalho2'] = dfrestante['atalho2'].apply(lambda x: tira3(x))

In [ ]:
ped_pub

In [ ]:
outros

In [ ]:
dfrestante

In [ ]:
dfrestante = dfrestante.reset_index(drop=True)
dfrestante

In [ ]:
# Extraindo somente o primeiro país e o estado, caso seja depositante nacional
b=ped_pub['atalho2']
origem_depos=[]
origem_depos_estado=[]
for i in b:
  if len(i)==2:
    nb=i
    nb1=0
  else:
    nb=i[0:2]
    if i[0:2]=='BR':
      nb1=i[-2:]
    elif i[0:2]!='BR':
        nb1=0
  origem_depos.append(nb)
  origem_depos_estado.append(nb1)
origem_depos=pd.DataFrame(origem_depos,columns=['ori_depos'])
origem_depos_estado=pd.DataFrame(origem_depos_estado,columns=['ori_depos_est'])
ped_pub['ori_depos']=origem_depos
frames=[ped_pub,origem_depos_estado]
ped_pub=pd.concat(frames, axis=1)


b=outros['atalho2']
origem_depos=[]
origem_depos_estado=[]
for i in b:
  if len(i)==2:
    nb=i
    nb1=0
  else:
    nb=i[0:2]
    if i[0:2]=='BR':
      nb1=i[-2:]
    elif i[0:2]!='BR':
        nb1=0
  origem_depos.append(nb)
  origem_depos_estado.append(nb1)
origem_depos=pd.DataFrame(origem_depos,columns=['ori_depos'])
origem_depos_estado=pd.DataFrame(origem_depos_estado,columns=['ori_depos_est'])
outros['ori_depos']=origem_depos
frames=[outros,origem_depos_estado]
outros=pd.concat(frames, axis=1)

b=dfrestante['atalho2']
origem_depos=[]
origem_depos_estado=[]
for i in b:
  if len(i)==2:
    nb=i
    nb1=0
  else:
    nb=i[0:2]
    if i[0:2]=='BR':
      nb1=i[-2:]
    elif i[0:2]!='BR':
      nb1=0
  origem_depos.append(nb)
  origem_depos_estado.append(nb1)
origem_depos=pd.DataFrame(origem_depos,columns=['ori_depos'])
origem_depos_estado=pd.DataFrame(origem_depos_estado,columns=['ori_depos_est'])
dfrestante['ori_depos']=origem_depos
frames=[dfrestante,origem_depos_estado]
dfrestante=pd.concat(frames, axis=1)

In [ ]:
ped_pub

In [ ]:
outros

In [ ]:
# Deletando as colunas atalho criadas
del ped_pub['atalho']
del ped_pub['atalho2']

del outros['atalho']
del outros['atalho2']

del dfrestante['atalho']
del dfrestante['atalho2']

In [ ]:
ped_pub

In [ ]:
outros

In [ ]:
dfrestante

Alterando as ordens da coluna e salvando em um novo dataframe:

In [ ]:
ped_pub_final=ped_pub[['chave_idpedido','idpedido','data_deposito','original','ipc1','titulo1','depositante1','ori_depos','ori_depos_est','despacho','data_rpi']]
outros_final=outros[['chave_idpedido','idpedido','data_deposito','depositante1','ori_depos','ori_depos_est','despacho','data_rpi']]
dfrestante=dfrestante[['chave_idpedido','idpedido','data_deposito','original','ipc1','titulo1','depositante1','ori_depos','ori_depos_est','despacho','data_rpi']]

In [ ]:
ped_pub_final.sort_values(by=['idpedido']).head(60)

In [ ]:
outros_final.sort_values(by=['idpedido']).head(60)

In [ ]:
ped_pub_final.sort_values(by=['idpedido']).tail(60)

In [ ]:
outros_final.sort_values(by=['idpedido']).tail(60)

In [ ]:
ped_pub_final.sort_values(by=['data_deposito']).head(60)

In [ ]:
ped_pub_final.sort_values(by=['data_deposito']).head(60)

In [ ]:
outros_final.sort_values(by=['data_deposito']).head(60)

In [ ]:
ped_pub_final.sort_values(by=['data_deposito']).tail(60)

In [ ]:
outros_final.sort_values(by=['data_deposito']).tail(60)

In [ ]:
ped_pub_final.sort_values(by=['data_rpi']).head(60)

In [ ]:
outros_final.sort_values(by=['despacho']).head(60)

In [ ]:
outros_final.sort_values(by=['despacho']).tail(60)

In [ ]:
outros_final.sort_values(by=['data_rpi']).head(60)

In [ ]:
ped_pub_final.sort_values(by=['data_rpi']).tail(60)

In [ ]:
outros_final.sort_values(by=['data_rpi']).tail(60)

Renumerando o index do dataframe para ficar em ordem:

In [ ]:
ped_pub_final = ped_pub_final.reset_index(drop=True)
outros_final = outros_final.reset_index(drop=True)
dfrestante = dfrestante.reset_index(drop=True)

In [ ]:
ped_pub_final

In [ ]:
outros_final

In [ ]:
dfrestante

Vamos eliminar as barras de todas as datas para poder padronizar um formato único:


In [ ]:
ped_pub_final['data_deposito1'] = ped_pub_final['data_deposito'].apply(lambda x: str(x).replace('/',''))
ped_pub_final['data_rpi1'] = ped_pub_final['data_rpi'].apply(lambda x: str(x).replace('/',''))

outros_final['data_deposito1'] = outros['data_deposito'].apply(lambda x: str(x).replace('/',''))
outros_final['data_rpi1'] = outros['data_rpi'].apply(lambda x: str(x).replace('/',''))



In [ ]:
ped_pub_final.sort_values(by=['data_deposito1']).head(60)

In [ ]:
ped_pub_final.sort_values(by=['data_deposito1']).tail(60)

In [ ]:
outros_final.sort_values(by=['data_deposito1']).head(60)

In [ ]:
outros_final.sort_values(by=['data_deposito1']).tail(60)

In [ ]:
# Removendo os espaços das strings das colunas que contém data
dfrestante['data_rpi'] = dfrestante['data_rpi'].apply(lambda x: str(x).replace(' ',''))
dfrestante['data_deposito']= dfrestante['data_deposito'].apply(lambda x: str(x).replace(' ',''))

In [ ]:
# Função para transformar a data no formato data do python:
from datetime import datetime
def data_f(coluna):
  data=[]
  ndata=''
  lista=[]
  for i in coluna:
    if len(str(i))==8:
      y=str(i[-4:])
      m=str(i[2:4])
      d=str(i[0:2])
      z=y+'-'+m+'-'+d
    elif len(str(i))==6:
      y='19'+str(i[-2:])
      m=str(i[2:4])
      d=str(i[0:2])
      if m.isnumeric()==False:
        m='10'
      elif d.isnumeric()==False:
        d='10'
      z=y+'-'+m+'-'+d
    elif len(str(i))<6 :
      z='1991-01-01'
    lista.append(z)

  for i in lista:
    try:
      ndata = datetime.strptime(i, '%Y-%m-%d').date()
    except:
      ndata = datetime.strptime('1991-01-01', '%Y-%m-%d').date()
    data.append(ndata)
  data=pd.DataFrame(data,columns=['data_'])
  return data

In [ ]:
# Função para transformar a data no formato data do python:
from datetime import datetime
def data_r(coluna):
  data=[]
  ndata=''
  lista=[]
  for i in coluna:
    if len(str(i))==10:
      y=str(i[-4:])
      m=str(i[3:5])
      d=str(i[0:2])
      z=y+'-'+m+'-'+d
    lista.append(z)
  for i in lista:
    ndata = datetime.strptime(i, '%Y-%m-%d').date()
    data.append(ndata)
  data=pd.DataFrame(data,columns=['data_'])
  return data

In [ ]:
# aplicando as funções
ped_pub_final['data_deposito2']=data_f(ped_pub_final['data_deposito1'])
ped_pub_final['data_rpi2']=data_f(ped_pub_final['data_rpi1'])

outros_final['data_deposito2']=data_f(outros_final['data_deposito1'])
outros_final['data_rpi2']=data_f(outros_final['data_rpi1'])


In [ ]:
dfrestante['data_deposito1']=data_r(dfrestante['data_deposito'])
dfrestante['data_rpi1']=data_r(dfrestante['data_rpi'])

In [ ]:
ped_pub_final

In [ ]:
outros_final

In [ ]:
dfrestante

In [ ]:
outros_final

In [ ]:
ped_pub_final=ped_pub_final[['chave_idpedido','idpedido','data_deposito2','original','ipc1','titulo1','depositante1','ori_depos','ori_depos_est','despacho','data_rpi2']]

In [ ]:
dfrestante=dfrestante[['chave_idpedido','idpedido','data_deposito1','original','ipc1','titulo1','depositante1','ori_depos','ori_depos_est','despacho','data_rpi1']]

In [ ]:
outros_f=outros_final[['chave_idpedido','idpedido','data_deposito2','depositante1','ori_depos','ori_depos_est','despacho','data_rpi2']]

In [ ]:
ped_pub_final.info()

In [ ]:
outros_f.info()

In [ ]:
outros_f['despacho'].unique()

In [ ]:
outros_f.columns = ['chave_idpedido','idpedido','data_deposito','depositante','ori_depos','ori_depos_est','despacho','data_rpi']

In [ ]:
outros_f

In [ ]:
import numpy as np
np.size(outros_f['depositante'].unique())

In [ ]:
np.size(outros_f['ori_depos'].unique())

In [ ]:
# Função para tirar as vírgulas dos depositantes
def tira4(test_str):
    saida = ''
    v1 = 0
    for i in str(test_str):
        if i == ',':
            v1 += 1
        elif i == ','  and v1>0:
            v1 -= 1
        elif v1 == 0:
            saida += i
    return saida

In [ ]:
outros_f['depositante2'] = outros_f['depositante'].apply(lambda x: tira4(x))
outros_f

Removendo os caracteres especiais dos depositantes:

In [ ]:
import re
outros_f['depositante2'] = outros_f['depositante2'].map(lambda x: re.sub(r'\W+', ' ', str(x)))
outros_f['depositante2']=outros_f['depositante2'].str.upper()
outros_f

Removendo a acentuação dos depositantes, para evitar diferenças:

In [ ]:
import numpy as np
outros_f['depositante2']=outros_f['depositante2'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
outros_f

In [ ]:
del outros_f['depositante']
outros_f=outros_f[['chave_idpedido','idpedido','data_deposito','depositante2','ori_depos','ori_depos_est','despacho','data_rpi']]
outros_f.columns = ['chave_idpedido','idpedido','data_deposito','depositante','ori_depos','ori_depos_est','despacho','data_rpi']
outros_f

In [ ]:
np.size(outros_f['depositante'].unique())

In [ ]:
# salvando como arquivo csv para trabalhar no arquivo posteriormente:
outros_f.to_csv("outros_f.csv", index=False)

Voltando para o dataframe ped_pub_final e dfrestante:

In [ ]:
ped_pub_final

In [ ]:
dfrestante

In [ ]:
ped_pub_final.sort_values(by=['data_deposito2']).head(60)

In [ ]:
ped_pub_final.sort_values(by=['data_deposito2']).tail(60)

Vamos separar as classificações em três níveis. O primeiro nível, é só a primeira letra da classificação i[0:1], o segundo nível trata dos 3 primeiros digitos da classificação i[0:3] e o terceiro nível de detalhe são os 4 primeiros dígitos i[0:4]. Lembrando que cada nível apresenta um detalhamento maior da tecnologia a que se refere aquela classificação:

In [ ]:
ped_pub_final['ipc1_1'] = ped_pub_final['ipc1'].apply(lambda x: x[0:1])
dfrestante['ipc1_1'] = dfrestante['ipc1'].apply(lambda x: x[0:1])

In [ ]:
ped_pub_final['ipc1_2'] = ped_pub_final['ipc1'].apply(lambda x: x[0:3])
dfrestante['ipc1_2'] = dfrestante['ipc1'].apply(lambda x: x[0:3])

In [ ]:
ped_pub_final['ipc1_3'] = ped_pub_final['ipc1'].apply(lambda x: x[0:4])
dfrestante['ipc1_3'] = dfrestante['ipc1'].apply(lambda x: x[0:4])

In [ ]:
ped_pub_final

In [ ]:
dfrestante

Removendo algumas repetições que ocorrem em algumas linhas da coluna depositante1:

In [ ]:
def tira4(test_str):
    saida = ''
    v1 = 0
    for i in str(test_str):
        if i == ',':
            v1 += 1
        elif i == ','  and v1>0:
            v1 -= 1
        elif v1 == 0:
            saida += i
    return saida

In [ ]:
ped_pub_final['depositante2'] = ped_pub_final['depositante1'].apply(lambda x: tira4(x))
dfrestante['depositante2'] = dfrestante['depositante1'].apply(lambda x: tira4(x))

In [ ]:
ped_pub_final

In [ ]:
dfrestante

In [ ]:
import re
ped_pub_final['depositante3'] = ped_pub_final['depositante2'].map(lambda x: re.sub(r'\W+', ' ', str(x)))
ped_pub_final['depositante3']=ped_pub_final['depositante3'].str.upper()

dfrestante['depositante3'] = dfrestante['depositante2'].map(lambda x: re.sub(r'\W+', ' ', str(x)))
dfrestante['depositante3']=dfrestante['depositante3'].str.upper()
ped_pub_final

In [ ]:
dfrestante

In [ ]:
import numpy as np
ped_pub_final['depositante4']=ped_pub_final['depositante3'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
dfrestante['depositante4']=dfrestante['depositante3'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
ped_pub_final

In [ ]:
dfrestante

In [ ]:
ped_pub_final['titulo2']=ped_pub_final['titulo1'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
dfrestante['titulo2']=dfrestante['titulo1'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
ped_pub_final

In [ ]:
dfrestante

In [ ]:
ped_pub_f=ped_pub_final[['chave_idpedido','idpedido','data_deposito2','original','ipc1','ipc1_1','ipc1_2','ipc1_3','titulo2','depositante4','ori_depos','ori_depos_est','despacho','data_rpi2']]
ped_pub_f.columns = ['chave_idpedido','idpedido','data_deposito','original','ipc','ipc_1','ipc_2','ipc_3','titulo','depositante','ori_depos','ori_depos_est','despacho','data_rpi']
dfrestante=dfrestante[['chave_idpedido','idpedido','data_deposito1','original','ipc1','ipc1_1','ipc1_2','ipc1_3','titulo2','depositante4','ori_depos','ori_depos_est','despacho','data_rpi1']]
dfrestante.columns = ['chave_idpedido','idpedido','data_deposito','original','ipc','ipc_1','ipc_2','ipc_3','titulo','depositante','ori_depos','ori_depos_est','despacho','data_rpi']

In [ ]:
ped_pub_f

In [ ]:
dfrestante

Verificando se há idpedidos repetidos:

In [ ]:
dupl=ped_pub_f['chave_idpedido'].value_counts().rename_axis('unique_values').to_frame('counts')
dupl2=dfrestante['chave_idpedido'].value_counts().rename_axis('unique_values').to_frame('counts')

In [ ]:
dupl.head(60)

In [ ]:
dupl2.head(60)

In [ ]:
print(ped_pub_f['ori_depos'].value_counts().rename_axis('unique_values').to_frame('counts').to_string())

In [ ]:
ped_pub_f['depositante'].value_counts().rename_axis('unique_values').to_frame('counts').head(60)

In [ ]:
ped_pub_f['depositante'].value_counts().rename_axis('unique_values').to_frame('counts').tail(60)

In [ ]:
ped_pub_f.loc[ped_pub_f['chave_idpedido'] == 'BR112014000556']

In [ ]:
ped_pub_f.loc[ped_pub_f['chave_idpedido'] == 'BR112014025730']

In [ ]:
ped_pub_f.loc[ped_pub_f['chave_idpedido'] == 'BR112015014844']

Como observado acima, podem haver simples repetições da publicação do pedido por causa da anulação da publicação anterior ou qualquer outro motivo para correção de outras informações. Assim, manteremos o registro com a última publicação na rpi, removendo os registros anteriores.

In [ ]:
ped_pub_f=ped_pub_f.sort_values(by=['chave_idpedido','data_rpi'])

In [ ]:
ped_pub_f = ped_pub_f.reset_index(drop=True)

In [ ]:
ped_pub_f

In [ ]:
ped_pub_f.loc[ped_pub_f['chave_idpedido'] == 'BR102017020904']

In [ ]:
ped_pub_f.loc[ped_pub_f['chave_idpedido'] == 'BR112014000976']

In [ ]:
ped_pub_f.loc[ped_pub_f['chave_idpedido'] == 'BR112015014844']

Eliminando, então, as linhas com idpedido repetidas, mantendo o último idpedido (data mais recente da rpi):

In [ ]:
ped_pub_f=ped_pub_f.drop_duplicates(subset='chave_idpedido', keep="last")

In [ ]:
ped_pub_f['chave_idpedido'].value_counts().rename_axis('unique_values').to_frame('counts')

Agora todas as idpedido repetidas foram removidas, mantendo-se apenas as que foram publicadas na rpi mais recente!

In [ ]:
ped_pub_f=ped_pub_f.sort_values(by=['data_deposito'])

In [ ]:
ped_pub_f = ped_pub_f.reset_index(drop=True)

In [ ]:
ped_pub_f

In [ ]:
ped_pub_f.to_csv("ped_pub_f.csv", index=False)

In [ ]:
dfrestante.to_csv('dfrestante_f.csv',index=False)

Vamos criar novas colunas descrevendo as tecnologias a que cada ipc representa de acordo com as definições da OMPI.

In [ ]:
!ls

In [ ]:
ped_pub_f=pd.read_csv("ped_pub_f.csv")

In [ ]:
dfrestante_f=pd.read_csv('dfrestante_f.csv')

In [ ]:
area_campo_ompi=pd.read_csv('area_campo_ompi.csv',encoding='iso-8859-1',delimiter=';')

In [ ]:
area_campo_ompi

In [ ]:
ped_pub_f=ped_pub_f.sort_values(by=['ipc'])
ped_pub_f = ped_pub_f.reset_index(drop=True)

dfrestante_f=dfrestante_f.sort_values(by=['ipc'])
dfrestante_f = dfrestante_f.reset_index(drop=True)

In [ ]:
ped_pub_f.head(60)

In [ ]:
ped_pub_f.tail(60)

In [ ]:
dfrestante_f

In [ ]:
verif=ped_pub_f.loc[ped_pub_f['ipc'].isna() == True].sort_values(by=['data_deposito'])
verif

In [ ]:
verif.loc[verif['data_deposito'] < '1993-01-01']

In [ ]:
verif.loc[verif['data_deposito'] > '2018-12-31']

Removendo os espaços entre as classificações:

In [ ]:
ped_pub_f['ipc'] = ped_pub_f['ipc'].str.replace(' ', '')
dfrestante_f['ipc'] = dfrestante_f['ipc'].str.replace(' ', '')

In [ ]:
ped_pub_f

In [ ]:
dfrestante_f

Inserindo uma nova coluna no df para utilizar as definições tecnologicas de arcodo com a OMPI:

In [ ]:
at=[]
n=''
for i in ped_pub_f['ipc']:
  i=str(i)
  if i[0:4]=='A61K':
    if i[0:6]=='A61K8/':
      n=i[0:6]
    else:
      n=i[0:4]
  elif i[0:4]=='B01D':
    if i[0:7]=='B01D45/' or i[0:7]=='B01D46/' or i[0:7]=='B01D47/' or i[0:7]=='B01D49/' or i[0:7]=='B01D50/' or i[0:7]=='B01D51/' or i[0:7]=='B01D52/':
      n=i[0:7]
    else:
        n=i[0:4]
  elif i[0:4]=='C13B':
    if i[0:6]=='C13B5/':
      n=i[0:6]
    elif i[0:7]=='C13B15/'  or i[0:7]=='C13B25/'  or i[0:7]=='C13B45/':
      n=i[0:7]
    else:
      n=i[0:4]
  elif i[0:4]=='E01F':
    if i[0:6]=='E01F8/':
      n=i[0:6]
    else:
      n=i[0:4]
  elif i[0:4]=='G01N':
    if i[0:7]=='G01N33/':
      n=i[0:7]
    else:
      n=i[0:4]
  elif i[0:4]=='H04N':
    if i[0:6]=='H04N1/':
      n=i[0:6]
    elif i[0:7]=='H04N21/':
      n=i[0:7]
    else:
      n=i[0:4]    
  else:
    n=i[0:4]
  at.append(n)
ipc_at=pd.DataFrame(at,columns=['ipc_at'])
ped_pub_f['ipc_at']=ipc_at

In [ ]:
at=[]
n=''
for i in dfrestante_f['ipc']:
  i=str(i)
  if i[0:4]=='A61K':
    if i[0:6]=='A61K8/':
      n=i[0:6]
    else:
      n=i[0:4]
  elif i[0:4]=='B01D':
    if i[0:7]=='B01D45/' or i[0:7]=='B01D46/' or i[0:7]=='B01D47/' or i[0:7]=='B01D49/' or i[0:7]=='B01D50/' or i[0:7]=='B01D51/' or i[0:7]=='B01D52/':
      n=i[0:7]
    else:
        n=i[0:4]
  elif i[0:4]=='C13B':
    if i[0:6]=='C13B5/':
      n=i[0:6]
    elif i[0:7]=='C13B15/'  or i[0:7]=='C13B25/'  or i[0:7]=='C13B45/':
      n=i[0:7]
    else:
      n=i[0:4]
  elif i[0:4]=='E01F':
    if i[0:6]=='E01F8/':
      n=i[0:6]
    else:
      n=i[0:4]
  elif i[0:4]=='G01N':
    if i[0:7]=='G01N33/':
      n=i[0:7]
    else:
      n=i[0:4]
  elif i[0:4]=='H04N':
    if i[0:6]=='H04N1/':
      n=i[0:6]
    elif i[0:7]=='H04N21/':
      n=i[0:7]
    else:
      n=i[0:4]    
  else:
    n=i[0:4]
  at.append(n)
ipc_at=pd.DataFrame(at,columns=['ipc_at'])
dfrestante_f['ipc_at']=ipc_at

In [ ]:
ped_pub_f

In [ ]:
dfrestante_f

Aplicando o método merge nos dois df, teremos uma nova coluna informando a área e o campo tecnológico de acordo com a tabela area_campo_ompi:

In [ ]:
result = pd.merge(ped_pub_f,area_campo_ompi, how='outer')
rdfrestante_f = pd.merge(dfrestante_f,area_campo_ompi, how='outer')

In [ ]:
result

In [ ]:
rdfrestante_f.head(60)

In [ ]:
rdfrestante_f.tail(60)

In [ ]:
result.head(60)

In [ ]:
result.tail(60)

Vamos guardar as linhas com NaN do df acima, talvez possa ser útil mais adiante. Depois vamos deletar estas linhas>

In [ ]:
nan_result=result[619157:]
nan_result

In [ ]:
nan_result.to_csv("nan_result.csv", index=False)

In [ ]:
result = result.drop(result[619157:].index)
result.tail(60)

In [ ]:
rdfrestante_f = rdfrestante_f.drop(rdfrestante_f[979:].index)
rdfrestante_f.tail(60)

Podemos criar mais uma coluna, informando se o depositante é de origem internacional (Não-Residente) ou nacional (Residente):

In [ ]:
result

In [ ]:
def nac_int(x):
  if x=='BR':
    y='residente'
  else:
    y='nao_residente'
  return (y)


In [ ]:
result['Nac_Inter']=result['ori_depos'].apply(lambda x: nac_int(x))
rdfrestante_f['Nac_Inter']=rdfrestante_f['ori_depos'].apply(lambda x: nac_int(x))
result

In [ ]:
rdfrestante_f

In [ ]:
result['Nac_Inter'].value_counts().rename_axis('unique_values').to_frame('counts')

In [ ]:
result['area_campo_ompi'].value_counts().rename_axis('unique_values').to_frame('counts')

In [ ]:
result=result.sort_values(by=['data_deposito'])
result = result.reset_index(drop=True)

In [ ]:
result.head(60)

In [ ]:
result.tail(60)

In [ ]:
result.loc[result['data_deposito'] < '1993-01-01']

In [ ]:
#result = result.drop(result[0:12455].index)

In [ ]:
result = result.reset_index(drop=True)
result

In [ ]:
result.loc[result['data_deposito'] >'2020-12-31']

In [ ]:
result.to_csv("result_f.csv", index=False)

In [ ]:
rdfrestante_f.to_csv("rdfrestante_f_f.csv", index=False)

In [ ]:
rdfrestante_f

In [ ]:
outros_f

In [ ]:
outros_f['despacho'].value_counts().rename_axis('unique_values').to_frame('counts')

Cada despacho significa:

11.1.1 - Arquivamento - Art. 33 Parágravo único da LPI

11.11 - Arquivamento - Art.17§ 2º da LPI

11.18 - Arquivamento definitivo por não anuência relacionada com o Art. 229-C da LPI.

11.2 - Arquivamento definitivo - Art. 36 §1° da LPI

11.4 - Arquivamento - Art.38 § 2º da LPI

11.5 - Arquivamento - Art.34 da LPI

11.6 - Arquivamento do Pedido - Art.216 § 2º da LPI

11.30 - Arquivamento Definitivo - Art. 18 § 1º da Lei 5772/71

11.31 - Arquivamento Definitivo - Falta de Cumprimento de exigência

11.6 - Arquivamento do Pedido - Art.216 § 2º da LPI

6.1 - Exigência - Art.36 da LPI

6.20 - Exigência Pré-Exame - Art. 34 da LPI

6.21 - Exigência preliminar - pedidos com buscas realizadas por outros Escritórios de Patentes

6.22 - Exigência preliminar - pedidos sem buscas realizadas por outros Escritórios de Patentes

7.1 - Conhecimento do Parecer Técnico

8.6 - Arquivamento - Art.86 da LPI

8.11 - Manutenção do Arquivamento

8.12 - Arquivamento Definitivo

9.1 - Deferimento

9.2 - Indeferimento



In [ ]:
outros_f.to_csv("outros_f.csv", index=False)

In [ ]:
# Removendo os espaços das colunas
nova_num_total_ed.columns = ['chave_original', 'chave_idpedido'] 
nova_num_total_ed['chave_original'] = nova_num_total_ed['chave_original'].str.replace(' ', '')
nova_num_total_ed['chave_idpedido'] = nova_num_total_ed['chave_idpedido'].str.replace(' ', '')

In [ ]:
nova_num_total_ed

Após as transformações acima, juntamos os pedidos restantes das RPI's 2118 e 2119 ao dataframe com as outras publicações. As verificações de duplicidade e verificação dos pedidos com renumeração foram feitas no excel, pelo método procv.

Primeiro, antes de fazer o merge, tivemos que preparar os arquivos para fazer o merge de forma correta. Dividimos o arquivo com os pedidos publicados em 2. Um, em que não houve renumeração (result_f_p1xm, para fazer um merge apenas) e outro (result_f_p2xm), contendo uma coluna a mais com a nova numeração do pedido. No arquivo em que temos somente pedidos que foram renumerados, teremos que fazer 2x o merge. No primeiro merge, faremos considerando a numeração antiga, na segunda, a nova numeração, e depois verificaremos via excel. Os arquivos já preparados, serão mostrados abaixo:

O arquivo que faremos o merge é o outros_f_pm, também preparado no excel, em que algumas verificações de data e informações foram feitas, bem como a exclusão dos pedidos com data de depósito inferior a 1993.

In [ ]:
import pandas as pd
outros_f_pm=pd.read_csv('outros_f_pm.csv',encoding='iso-8859-1',delimiter=';')
result_f_p1xm=pd.read_csv('result_f_p1xm.csv',encoding='iso-8859-1',delimiter=';')
result_f_p2xm=pd.read_csv('result_f_p2xm.csv',encoding='iso-8859-1',delimiter=';')

In [ ]:
outros_f_pm

In [ ]:
result_f_p1xm

In [ ]:
result_f_p2xm

Vamos juntar os dois dataframes utilizando o método merge:

In [ ]:
def e_desp(df):
  df1=[]
  c=['6.1','6.20','6.21','6.22','7.1','8.6','8.11','8.12','9.1','9.2','11.1.1','11.2','11.4','11.6','11.11','11.18','11.30','11.31']
  for desp in c:
    at=df.loc[df['despacho'] == desp].sort_values(by=['chave_idpedido','data_rpi'])
    at = at.reset_index(drop=True)
    at['chave_idpedido'].value_counts()
    at[desp] = at.groupby('chave_idpedido')['chave_idpedido'].transform('count')
    at=at.drop_duplicates(subset='chave_idpedido', keep="last")
    del at['despacho']
    at=at[['chave_idpedido',desp,'data_rpi']]
    at = at.reset_index(drop=True)
    df1.append(at)

  return df1[0], df1[1], df1[2], df1[3], df1[4], df1[5], df1[6], df1[7], df1[8], df1[9], df1[10], df1[11], df1[12], df1[13], df1[14], df1[15], df1[16], df1[17]

In [ ]:
df_6_1, df_6_20, df_6_21, df_6_22, df_7_1, df_8_6, df_8_11, df_8_12, df_9_1, df_9_2, df_11_1_1, df_11_2,df_11_4,df_11_6,df_11_11,df_11_18,df_11_30,df_11_31=e_desp(outros_f_pm)

In [ ]:
a=[df_6_1,df_6_20,df_6_21,df_6_22,df_7_1,df_8_6,df_8_11,df_8_12,df_9_1,df_9_2,df_11_1_1,df_11_2,df_11_4,df_11_6,df_11_11,df_11_18,df_11_30,df_11_31]
a

In [ ]:
for i in a:
  result_f_p1xm=pd.merge(result_f_p1xm,i,how='outer',on=['chave_idpedido'])
  result_f_p2xm=pd.merge(result_f_p2xm,i,how='outer',on=['chave_idpedido']) # primeiro merge, a chave idpedido do dataframe result_f_p2xm está ativada na 1º coluna

In [ ]:
result_f_p1xm

In [ ]:
result_f_p2xm

In [ ]:
result_f_p2xm = result_f_p2xm.drop(result_f_p2xm[1303:].index)

Alterando a chave do result_f_p2xm para a nova numeração, pela simples alteração do nome da coluna e refazendo o merge:

In [ ]:
result_f_p2xm.rename(columns={'chave_idpedido': 'num_anterior', 'nova_num': 'chave_idpedido'}, inplace=True)

In [ ]:
result_f_p2xm

In [ ]:
for i in a:
  result_f_p2xm=pd.merge(result_f_p2xm,i,how='outer',on=['chave_idpedido']) # segundo merge, a chave idpedido do dataframe result_f_p2xm está ativada na coluna com a nova numeração

In [ ]:
result_f_p2xm = result_f_p2xm.drop(result_f_p2xm[1303:].index)
result_f_p2xm.rename(columns={'num_anterior':'chave_idpedido', 'chave_idpedido':'nova_num'}, inplace=True)
result_f_p2xm

Este dataframe completo, contendo os dados bibliográficos dos pedidos e as ações que estes pedidos sofreram ao longo do tempo não terá utilidade neste trabalho específico, mas somente para uso interno do INPI.
Os dados utilizados serão os dados compilados dos dataframes que contém somente os dados bibliográficos e do dataframe outros.

In [ ]:
dupl=result_f_p1xm['chave_idpedido'].value_counts().rename_axis('unique_values').to_frame('counts')
dupl

In [ ]:
dupl=result_f_p2xm['chave_idpedido'].value_counts().rename_axis('unique_values').to_frame('counts')
dupl

In [ ]:
result_f_p1xm.to_csv("result_f_1.csv", index=False)
result_f_p2xm.to_csv("result_f_2.csv", index=False)

Terminamos a segunda etapa! Vamos à terceira etapa do trabalho.

In [ ]:
from google.colab import files
files.download("result_f_1.csv") 
files.download("result_f_2.csv") 

In [ ]:
from google.colab import files
files.download("outros_f.csv") 

# **TERCEIRA ETAPA - Análise, exploração e representação dos dados**

Neste etapa será possível encontrar padrões e tendências sobre o depósito de patentes no Brasil de distintas áreas tecnológicas.

Houve a necessidade de reagrupar os dados no excel e padronizar ao menos 150 nomes de depositantes, para contagem correta dos depositantes.

Além disso, foram corrigidas algumas datas e excluídos os pedidos depositados antes de 1993. Sendo os dados estudados daqui em diante no período de 1993 a 2016. 

Vamos aos gráficos:

In [ ]:
!ls

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation

In [ ]:
# Lendo o arquivo Dados_Final.csv
Dados_Final=pd.read_csv('Dados_Final.csv',encoding='iso-8859-1',delimiter=';')
Dados_Final

In [ ]:
Dados_Final['depositante'].value_counts().rename_axis('unique_values').to_frame('counts')

Total de despachos de interesse da Diretoria de Patentes, por ano no Brasil:

In [ ]:
# Lendo o arquivo despachoxano.csv
despachos_ano=pd.read_csv('despachosxano.csv',encoding='iso-8859-1',delimiter=';')
despachos_ano

In [ ]:
# substituindo os valores nan por 0 no dataframe
despachos_ano=despachos_ano.fillna(0)
despachos_ano

Os dados de despachos podem ver visualizados no período de 1993 a 2020 (até 08/12/2020), sendo que estas publicações demonstram, efetivamente, o que o INPI produziu até a data de publicação da RPI.
Já no caso da publicação de pedidos de patentes, há um risco de ter pedidos PCT ainda não analisados pela área designada para tal tarefa, por isso utilizaremos os dados no período de 1993 a 2016.

In [ ]:
# altera a fonte dos eixos
ft=12
plt.rcParams.update({'font.size':ft})

# mostra o tamanho default do gráfico
fig_size=plt.rcParams['figure.figsize']
fig_size

# altera o tamanho do gráfico
fig_size[0]=18
fig_size[1]=10

for ind in despachos_ano.index:
  plt.plot(despachos_ano.iloc[ind][1::])
plt.legend(despachos_ano['Despachos'].values)
plt.xlabel('Ano')
plt.ylabel('Despacho publicados')
plt.show()


Pode-se notar que o número total de despachos, ditos despachos de decisão (9.1 - Deferimento, 9.2 - Indeferimento, 11.2 - Arquivamento definitivo e 11.5 - Arquivamento) começaram a ter um aumento significativo a partir de 2016. O que evidencia um aumento significativo no número de decisões da Diretoria de Patentes. Uma das razões para este aumento na produtividade da DIRPA tem relação direta com o plano de ataque ao BACKLOG do INPI, bem como ao aumento significativo na quantidade de pedidos de patentes analisados pelos examinadores de patentes do Instituto.

Pedidos de patentes de residentes (nacionais) e não residentes (internacionais).

In [ ]:
# Lendo o arquivo despachoxano.csv
res_nao_res_ano=pd.read_csv('res_nao_resxano.csv',encoding='iso-8859-1',delimiter=';')
res_nao_res_ano

In [ ]:
for ind in res_nao_res_ano.index:
  plt.plot(res_nao_res_ano.iloc[ind][1::])
plt.legend(res_nao_res_ano['Ano'].values)
plt.xlabel('Ano')
plt.ylabel('Número de pedidos de patentes publicados')
plt.show()

Pela evolução do gráfico, é notória a diferença entre o depósito de pedidos de patentes nacional e internacional. Observa-se que entre 1993 e 1995 a diferença entre os depósitos nacionais e internacionais era bem menor quando compara-se esta diferença, por exemple, em 2013. Inúmeras explicações podem ser dadas a este fato. Fatores sócio-econômicos, investimento em ciência/inovação e tratados internacionais podem ser citados para explicar tal discrepância, mas foge do escopo do presente trabalho.

Destes depositantes internacionais, quais os países com maior número de pedidos ao longo dos anos? Quais os estados brasileiros que mais depositam patentes?
Veremos nos próximos gráficos.

Número total de depósitos de pedidos por país por ano:

In [ ]:
# Importando bibliotecas para gráfico de barras horizontal
import matplotlib.ticker as ticker
import matplotlib.animation as animation
from IPython.display import HTML

In [ ]:
# Lendo o arquivo despachoxano.csv
pais_ano=pd.read_csv('paisxano.csv',encoding='iso-8859-1',delimiter=';')
pais_ano

In [ ]:
# substituindo os valores nan por 0 no dataframe
pais_ano=pais_ano.fillna(0)
pais_ano

In [ ]:
# Pegando apenas o ano de 2016
current_Ano = 2016
pais_ano_1 = (pais_ano[pais_ano['Ano'].eq(current_Ano)]
       .sort_values(by='Depósitos', ascending=False).head(10))
pais_ano_1

In [ ]:
#plotando no gráfico de barras horizontal
fig, ax = plt.subplots(figsize=(15, 8))
ax.barh(pais_ano_1['País'], pais_ano_1['Depósitos'])

In [ ]:
# Vamos adicionar cores de acordo com o continente de cada país
colors = dict(zip(
    ['Oceania', 'Europe', 'Asia', 'Latin America', 'North America'],
    ['#adb0ff', '#aafbff', '#90d595', '#e48381', '#f7bb5f']
))
group_lk = pais_ano.set_index('País')['Continente'].to_dict()

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))
pais_ano_1 = pais_ano_1[::-1]   # flip values from top to bottom
# pass colors values to `color=`
ax.barh(pais_ano_1['País'], pais_ano_1['Depósitos'], color=[colors[group_lk[x]] for x in pais_ano_1['País']])
# iterate over the values to plot labels and values
for i, (Depósitos, País) in enumerate(zip(pais_ano_1['Depósitos'], pais_ano_1['País'])):
    ax.text(Depósitos, i,     País,            ha='right')  # País: name
    ax.text(Depósitos, i-.25, group_lk[País],  ha='right')  # Asia: Continente
    ax.text(Depósitos, i,     Depósitos,           ha='left')   # Depósitos
# Add year right middle portion of canvas
ax.text(1, 0.4, current_Ano, transform=ax.transAxes, size=46, ha='right')

In [ ]:
# Melhorando a apresentação do gráfico
fig, ax = plt.subplots(figsize=(15, 8))
def draw_barchart(ano):
    pais_ano_1 = pais_ano[pais_ano['Ano'].eq(ano)].sort_values(by='Depósitos', ascending=True).tail(10)
    ax.clear()
    ax.barh(pais_ano_1['País'], pais_ano_1['Depósitos'], color=[colors[group_lk[x]] for x in pais_ano_1['País']])
    dx = pais_ano_1['Depósitos'].max() / 200
    for i, (Depósitos, País) in enumerate(zip(pais_ano_1['Depósitos'], pais_ano_1['País'])):
        ax.text(Depósitos-dx, i,     País,           size=14, weight=600, ha='right', va='bottom')
        ax.text(Depósitos-dx, i-.25, group_lk[País], size=10, color='#444444', ha='right', va='baseline')
        ax.text(Depósitos+dx, i,     f'{Depósitos:,.0f}',  size=14, ha='left',  va='center')
    # ... polished styles
    ax.text(1, 0.4, ano, transform=ax.transAxes, color='#777777', size=46, ha='right', weight=800)
    ax.xaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
    ax.xaxis.set_ticks_position('top')
    ax.tick_params(axis='x', colors='#777777', labelsize=12)
    ax.set_yticks([])
    ax.margins(0, 0.01)
    ax.grid(which='major', axis='x', linestyle='-')
    ax.set_axisbelow(True)
    ax.text(0, 1.12, 'Países com maior número de depósitos de pedidos de patente no Brasil',
            transform=ax.transAxes, size=24, weight=600, ha='left')
    plt.box(False)
    
draw_barchart(2016)

In [ ]:
# Finalmente, utilizando o recurso de animação do matplotlib
import matplotlib.animation as animation
from IPython.display import HTML
fig, ax = plt.subplots(figsize=(18, 10))
animator = animation.FuncAnimation(fig, draw_barchart, frames=range(1993, 2017))
HTML(animator.to_jshtml()) 
# or use animator.to_html5_video() or animator.save()

In [ ]:
animator.save('pais_ano.mp4')

In [ ]:
from google.colab import files
files.download('pais_ano.mp4') 

Pela visualização do vídeo 1, podemos observar que no período de 1993 a 1997 o Brasil era o que mantinha o maior número de depósitos de pedidos de patente em seu território. Em 1998 e 1999 os Estados Unidos assumiram a liderança e, de 1999 a 2005, o Brasil voltou a ser o primeiro em número de depósitos. Após 2005, os Estados Unidos seguiram como os maiores depositantes em território brasileiro, se afastando cada vez dos depositantes nacionais, e de outros países, até 2016. Como pode ser visto na figura 3. Também é notório que o continente europeu é maioria entre os 10 maiores depositantes, independente do ano. O Japão seguia isolado como representante asiático até o ano de 2012, sendo que a partir de 2013 a China entra para seleto grupo dos top 10.

Outra pergunta interessante que podemos responder com os dados compilados é: Qual empresa possuir maior depósito de pedidos de patentes?

Depósitos de pedidos de patentes de empresas por ano:

In [ ]:
# Lendo o arquivo despachoxano.csv
dep_ano=pd.read_csv('depositantexano.csv',encoding='iso-8859-1',delimiter=';')

# substituindo os valores nan por 0 no dataframe
dep_ano=dep_ano.fillna(0)
dep_ano

In [ ]:
# Verificando quais os países contém os dados
pais_un=dep_ano['País'].value_counts().rename_axis('unique_values').to_frame('counts')
pais_un

In [ ]:
# Pegando apenas o ano de 2016
current_Ano = 2016
dep_ano_1 = (dep_ano[dep_ano['Ano'].eq(current_Ano)]
       .sort_values(by='Depósitos', ascending=False).head(10))
dep_ano_1

In [ ]:
# Vamos adicionar cores de acordo com cada país
colors = dict(zip(
    ['United States', 'Deutschland', 'Netherlands', 'Switzerland', 'France','Korea','Japan','Sweden'],
    ['#D84E4E', '#aafbff', '#90d595', '#e48381', '#f7bb5f','#1f77b4','#e377c2','#9467bd']
))
group_lk = dep_ano.set_index('Depositante')['País'].to_dict()
group_lk

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))
dep_ano_1 = dep_ano_1[::-1]   # flip values from top to bottom
# pass colors values to `color=`
ax.barh(dep_ano_1['Depositante'], dep_ano_1['Depósitos'], color=[colors[group_lk[x]] for x in dep_ano_1['Depositante']])
# iterate over the values to plot labels and values
for i, (Depósitos, Depositante) in enumerate(zip(dep_ano_1['Depósitos'], dep_ano_1['Depositante'])):
    #ax.text(Depósitos, i,     Depositante,            ha='right')  
    ax.text(Depósitos, i-.25, group_lk[Depositante],  ha='right')  
    ax.text(Depósitos, i,     Depósitos,           ha='left')   
# Add year right middle portion of canvas
ax.text(1, 0.4, current_Ano, transform=ax.transAxes, size=46, ha='right')

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))
def draw_barchart(ano):
    dep_ano_1 = dep_ano[dep_ano['Ano'].eq(ano)].sort_values(by='Depósitos', ascending=True).tail(10)
    ax.clear()
    ax.barh(dep_ano_1['Depositante'], dep_ano_1['Depósitos'], color=[colors[group_lk[x]] for x in dep_ano_1['Depositante']])
    dx = dep_ano_1['Depósitos'].max() / 200
    for i, (Depósitos, Depositante) in enumerate(zip(dep_ano_1['Depósitos'], dep_ano_1['Depositante'])):
        ax.text(Depósitos-dx, i,     Depositante,           size=14, weight=600, ha='right', va='bottom')
        ax.text(Depósitos-dx, i-.25, group_lk[Depositante], size=12, ha='right', va='baseline')
        ax.text(Depósitos+dx, i,     f'{Depósitos:,.0f}',  size=14, ha='left',  va='center')
    # ... polished styles
    ax.text(1, 0.4, ano, transform=ax.transAxes, color='#777777', size=46, ha='right', weight=800)
    ax.xaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
    ax.xaxis.set_ticks_position('top')
    ax.tick_params(axis='x', colors='#777777', labelsize=12)
    ax.set_yticks([])
    ax.margins(0, 0.01)
    ax.grid(which='major', axis='x', linestyle='-')
    ax.set_axisbelow(True)
    ax.text(0, 1.12, 'Empresas com maior número de depósitos de pedidos de patente no Brasil',
            transform=ax.transAxes, size=24, weight=600, ha='left')
    plt.box(False)
    
draw_barchart(2016)

In [ ]:
# Finalmente, utilizando o recurso de animação do matplotlib
import matplotlib.animation as animation
from IPython.display import HTML
fig, ax = plt.subplots(figsize=(16, 8))
animator = animation.FuncAnimation(fig, draw_barchart, frames=range(1993, 2017))
HTML(animator.to_jshtml()) 
# or use animator.to_html5_video() or animator.save()

In [ ]:
animator.save('depositante_ano.mp4')

In [ ]:
from google.colab import files
files.download('depositante_ano.mp4') 


A empresa com maior número de pedidos de patentes no Brasil, no período de 1993 a 2000 foi a Procter & Gamble, empresa americana de bens consumo que abrange diversos produtoas, dentre os quais se destacam os de higiene pessoal. Em 2001 foi a vez da Bosch dominar o depósito de pedidos de patentes no Brasil, empresa notadamente conhecida por soluções automotivas e máquinas ferramentas. Nos anos de 2002, 2003, 2006 a 2009 e 2014 a 2016, a Qualcomm dominou o primeiro lugar no depósito de pedidos de patentes, isto é, nove anos de domínio de inovação no país. A Qualcomm é uma empresa destacadamente conhecida no ramo da telecomunicação. Em 2004 e 2005 o destaque foi para as companhias Basf e Microsoft. A Basf, empresa bastente conhecida na área química, atua em diversas áreas, fornecendo soluções diversas para inúmeras áreas. A Microsoft, dispensa comentários. Já no período de 2010 a 2013 a Philips emplacou o primeiro lugar no depósito de pedidos de patentes. Empresa conhecida por produtos eletrônicos voltados para a área de higiene pessoal, setor automotivo e acessórios diversos.   

Com os dados das RPI's, foi possível saber, quantitativamente, quais os países com maior número de depósito de pedidos de patentes e quais as empresas dominam o cenário geral de inovação no Brasil. Podemos perguntar também, quais os estados brasileiros possuem o maior número de depósito de pediddos de patentes? E quais as empresas nacionais são destaques em inovação?
Veremos a seguir.

Depósitos de pedidos de patentes por estado:

In [ ]:
# Lendo o arquivo despachoxano.csv
est_ano=pd.read_csv('estadoxano.csv',encoding='iso-8859-1',delimiter=';')

# substituindo os valores nan por 0 no dataframe
est_ano=est_ano.fillna(0)
est_ano

In [ ]:
# Pegando apenas o ano de 2016
current_Ano = 2016
est_ano_1 = (est_ano[est_ano['Ano'].eq(current_Ano)]
       .sort_values(by='Depósitos', ascending=False).head(10))
est_ano_1

In [ ]:
# Vamos adicionar cores de acordo com cada país
colors = dict(zip(
    ['Sudeste', 'Sul', 'Nordeste', 'Norte', 'Centro-Oeste'],
    ['#D84E4E', '#aafbff', '#90d595', '#e48381', '#f7bb5f']
))
group_lk = est_ano.set_index('Estado')['Região'].to_dict()
group_lk

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))
est_ano_1 = est_ano_1[::-1]   # flip values from top to bottom
# pass colors values to `color=`
ax.barh(est_ano_1['Estado'], est_ano_1['Depósitos'], color=[colors[group_lk[x]] for x in est_ano_1['Estado']])
# iterate over the values to plot labels and values
for i, (Depósitos, País) in enumerate(zip(est_ano_1['Depósitos'], est_ano_1['Estado'])):
    ax.text(Depósitos, i,     País,            ha='right')  # País: name
    ax.text(Depósitos, i-.25, group_lk[País],  ha='right')  # Asia: Continente
    ax.text(Depósitos, i,     Depósitos,           ha='left')   # Depósitos
# Add year right middle portion of canvas
ax.text(1, 0.4, current_Ano, transform=ax.transAxes, size=46, ha='right')

In [ ]:
# Melhorando a apresentação do gráfico
fig, ax = plt.subplots(figsize=(15, 8))
def draw_barchart(ano):
    est_ano_1 = est_ano[est_ano['Ano'].eq(ano)].sort_values(by='Depósitos', ascending=True).tail(10)
    ax.clear()
    ax.barh(est_ano_1['Estado'], est_ano_1['Depósitos'], color=[colors[group_lk[x]] for x in est_ano_1['Estado']])
    dx = est_ano_1['Depósitos'].max() / 200
    for i, (Depósitos, País) in enumerate(zip(est_ano_1['Depósitos'], est_ano_1['Estado'])):
        ax.text(Depósitos-dx, i,     País,           size=14, weight=600, ha='right', va='bottom')
        ax.text(Depósitos-dx, i-.25, group_lk[País], size=10, color='#444444', ha='right', va='baseline')
        ax.text(Depósitos+dx, i,     f'{Depósitos:,.0f}',  size=14, ha='left',  va='center')
    # ... polished styles
    ax.text(1, 0.4, ano, transform=ax.transAxes, color='#777777', size=46, ha='right', weight=800)
    ax.xaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
    ax.xaxis.set_ticks_position('top')
    ax.tick_params(axis='x', colors='#777777', labelsize=12)
    ax.set_yticks([])
    ax.margins(0, 0.01)
    ax.grid(which='major', axis='x', linestyle='-')
    ax.set_axisbelow(True)
    ax.text(0, 1.12, 'Estado com maior número de depósitos de pedidos de patente no Brasil',
            transform=ax.transAxes, size=24, weight=600, ha='left')
    plt.box(False)
    
draw_barchart(2016)

In [ ]:
# Finalmente, utilizando o recurso de animação do matplotlib
import matplotlib.animation as animation
from IPython.display import HTML
fig, ax = plt.subplots(figsize=(16, 8))
animator = animation.FuncAnimation(fig, draw_barchart, frames=range(1993, 2017))
HTML(animator.to_jshtml()) 
# or use animator.to_html5_video() or animator.save()

In [ ]:
animator.save('estado_ano.mp4')

In [ ]:
from google.colab import files
files.download('estado_ano.mp4')

É notório a hegemonia das regiões sudeste e sul no cenário de inovação brasileiro. Destaque para os estados de São Paulo (desde 1993 está em primeiro lugar nos top 10), Rio de Janeiro, Rio Grande do Sul, Minas Gerais, Paraná e Santa Catarina.

Estas informações são consistentes, pois como citado em estudos anteriores, há uma forte correlação entre o PIB e o número de depósitos de pedidos de patentes. Estes estados que ocupam os 6 primeiros lugares desde 1993 são os estados com o maior PIB do Brasil, conforme levantamento do IBGE (https://www.ibge.gov.br/explica/pib.php).

São Paulo é o primeiro por questões óbvias, pois é a capital econômica do país, sendo o estado com maior número de empresas e concentrador da maior parte da pesquisa científica do país.

Maiores depositantes de pedidos de patentes nacionais:

In [ ]:
import pandas as pd
# Lendo o arquivo despachoxano.csv
dep_res_ano=pd.read_csv('depositante_residentexano.csv',encoding='iso-8859-1',delimiter=';')

# substituindo os valores nan por 0 no dataframe
dep_res_ano=dep_res_ano.fillna(0)
dep_res_ano

In [ ]:
# Pegando apenas o ano de 2016
current_Ano = 2016
dep_res_ano_1 = (dep_res_ano[dep_res_ano['Ano'].eq(current_Ano)]
       .sort_values(by='Depósitos', ascending=False).head(10))
dep_res_ano_1

In [ ]:
# Vamos adicionar cores de acordo com cada grupo (como são 2400 linhas, dividimos em 5 partes - 240 primeiros para o grupo A, após classificar por ordem alfabética, B para os próximos 240 e assim sucessivamente)
colors = dict(zip(
    ['A', 'B', 'C', 'D', 'E'],
    ['#D84E4E', '#aafbff', '#90d595', '#e48381', '#f7bb5f']
))
group_lk = dep_res_ano.set_index('Depositante')['Grupo'].to_dict()
group_lk

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))
dep_res_ano_1= dep_res_ano_1[::-1]   # flip values from top to bottom
# pass colors values to `color=`
ax.barh(dep_res_ano_1['Depositante'], dep_res_ano_1['Depósitos'], color=[colors[group_lk[x]] for x in dep_res_ano_1['Depositante']])
# iterate over the values to plot labels and values
for i, (Depósitos, Depositante) in enumerate(zip(dep_res_ano_1['Depósitos'], dep_res_ano_1['Depositante'])):
    #ax.text(Depósitos, i,     Depositante,            ha='right')  
    ax.text(Depósitos, i-.25, group_lk[Depositante],  ha='right')  
    ax.text(Depósitos, i,     Depósitos,           ha='left')   
# Add year right middle portion of canvas
ax.text(1, 0.4, current_Ano, transform=ax.transAxes, size=46, ha='right')

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))
def draw_barchart(ano):
    dep_res_ano_1 = dep_res_ano[dep_res_ano['Ano'].eq(ano)].sort_values(by='Depósitos', ascending=True).tail(10)
    ax.clear()
    ax.barh(dep_res_ano_1['Depositante'], dep_res_ano_1['Depósitos'], color=[colors[group_lk[x]] for x in dep_res_ano_1['Depositante']])
    dx = dep_res_ano_1['Depósitos'].max() / 200
    for i, (Depósitos, Depositante) in enumerate(zip(dep_res_ano_1['Depósitos'], dep_res_ano_1['Depositante'])):
        ax.text(Depósitos-dx, i,     Depositante,           size=14, weight=600, ha='right', va='bottom')
        ax.text(Depósitos-dx, i-.25, group_lk[Depositante], size=12, ha='right', va='baseline')
        ax.text(Depósitos+dx, i,     f'{Depósitos:,.0f}',  size=14, ha='left',  va='center')
    # ... polished styles
    ax.text(1, 0.4, ano, transform=ax.transAxes, color='#777777', size=46, ha='right', weight=800)
    ax.xaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
    ax.xaxis.set_ticks_position('top')
    ax.tick_params(axis='x', colors='#777777', labelsize=12)
    ax.set_yticks([])
    ax.margins(0, 0.01)
    ax.grid(which='major', axis='x', linestyle='-')
    ax.set_axisbelow(True)
    ax.text(0, 1.12, 'Empresas residentes no BR com maior n.º de pedidos de patente no BR',
            transform=ax.transAxes, size=24, weight=600, ha='left')
    plt.box(False)
    
draw_barchart(2016)

In [ ]:
# Finalmente, utilizando o recurso de animação do matplotlib
import matplotlib.animation as animation
from IPython.display import HTML
fig, ax = plt.subplots(figsize=(16, 8))
animator = animation.FuncAnimation(fig, draw_barchart, frames=range(1993, 2017))
HTML(animator.to_jshtml()) 
# or use animator.to_html5_video() or animator.save()

As empresas residentes que mostram maior número de pedidos de patentes no Brasil ao longo dos anos são notadamente a Petrobrás, CSN, Cia Vale do Rio Doce, Usiminas e Whirlpool. Além destas empresas, as universidades públicas também estão entres os 10 ao longo dos anos, cito: Unicamp, UFMG, USP, UFRJ, UFPR, UFRGS dentre outras federais. Interesse notar que em 2016, dos 10 maiores depositantes, 9 são universidades!

Sabemos quais empresas e países possuem o maior número de pedidos de patentes no Brasil. Agora, quais tecnologias são de maior interesse dos 2 países com maior número de depositos no INPI? Isto é, quais são as tecnologias de maior interesse destes países?


In [ ]:
animator.save('residente_ano.mp4')

In [ ]:
from google.colab import files
files.download('residente_ano.mp4')

Evolução das tecnologias, dos Estados Unidos,  com maior número de depósitos de pedidos de patente no Brasil:

In [ ]:
import pandas as pd
# Lendo o arquivo despachoxano.csv
ompi_US_ano=pd.read_csv('ompi_USxano.csv',encoding='iso-8859-1',delimiter=';')

# substituindo os valores nan por 0 no dataframe
ompi_US_ano=ompi_US_ano.fillna(0)
ompi_US_ano

In [ ]:
# Pegando apenas o ano de 2016
current_Ano = 2016
ompi_US_ano_1 = (ompi_US_ano[ompi_US_ano['Ano'].eq(current_Ano)]
       .sort_values(by='Depósitos', ascending=False).head(10))
ompi_US_ano_1

In [ ]:
# Vamos adicionar cores de acordo com a área tecnologica (Química, Instrumentos, Engenharia mecânica, Engenharia elétrica, Outras áreas - divisão esta utilizada pela WIPO)
colors = dict(zip(
    ['Química', 'Instrumentos', 'Engenharia mecânica', 'Engenharia elétrica', 'Outras áreas'],
    ['#D84E4E', '#aafbff', '#90d595', '#e48381', '#f7bb5f']
))
group_lk = ompi_US_ano.set_index('Campo_Tec')['Area_Tec'].to_dict()

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))
ompi_US_ano_1= ompi_US_ano_1[::-1]   # flip values from top to bottom
# pass colors values to `color=`
ax.barh(ompi_US_ano_1['Campo_Tec'], ompi_US_ano_1['Depósitos'], color=[colors[group_lk[x]] for x in ompi_US_ano_1['Campo_Tec']])
# iterate over the values to plot labels and values
for i, (Depósitos, Depositante) in enumerate(zip(ompi_US_ano_1['Depósitos'], ompi_US_ano_1['Campo_Tec'])):
    #ax.text(Depósitos, i,     Depositante,            ha='right')  
    ax.text(Depósitos, i-.25, group_lk[Depositante],  ha='right')  
    ax.text(Depósitos, i,     Depósitos,           ha='left')   
# Add year right middle portion of canvas
ax.text(1, 0.4, current_Ano, transform=ax.transAxes, size=46, ha='right')

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))
def draw_barchart(ano):
    ompi_US_ano_1 = ompi_US_ano[ompi_US_ano['Ano'].eq(ano)].sort_values(by='Depósitos', ascending=True).tail(10)
    ax.clear()
    ax.barh(ompi_US_ano_1['Campo_Tec'], ompi_US_ano_1['Depósitos'], color=[colors[group_lk[x]] for x in ompi_US_ano_1['Campo_Tec']])
    dx = ompi_US_ano_1['Depósitos'].max() / 200
    for i, (Depósitos, Depositante) in enumerate(zip(ompi_US_ano_1['Depósitos'], ompi_US_ano_1['Campo_Tec'])):
        ax.text(Depósitos-dx, i,     Depositante,           size=14, weight=600, ha='right', va='bottom')
        ax.text(Depósitos-dx, i-.25, group_lk[Depositante], size=12, ha='right', va='baseline')
        ax.text(Depósitos+dx, i,     f'{Depósitos:,.0f}',  size=14, ha='left',  va='center')
    # ... polished styles
    ax.text(1, 0.4, ano, transform=ax.transAxes, color='#777777', size=46, ha='right', weight=800)
    ax.xaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
    ax.xaxis.set_ticks_position('top')
    ax.tick_params(axis='x', colors='#777777', labelsize=12)
    ax.set_yticks([])
    ax.margins(0, 0.01)
    ax.grid(which='major', axis='x', linestyle='-')
    ax.set_axisbelow(True)
    ax.text(0, 1.12, 'Tecnologias, de residentes dos US, com maior núm. de pedidos no INPI',
            transform=ax.transAxes, size=24, weight=600, ha='left')
    plt.box(False)
    
draw_barchart(2016)

In [ ]:
# Utilizando o recurso de animação do matplotlib
import matplotlib.animation as animation
from IPython.display import HTML
fig, ax = plt.subplots(figsize=(16, 8))
animator = animation.FuncAnimation(fig, draw_barchart, frames=range(1993, 2017))
HTML(animator.to_jshtml()) 
# or use animator.to_html5_video() or animator.save()

In [ ]:
animator.save('ompi_US_ano.mp4')

In [ ]:
from google.colab import files
files.download('ompi_US_ano.mp4')

Observa-se que na década de 1990 até 2010, a área tecnológica da química ocupou os três primeiros lugares, seguida da área de instrumentos, engenharia mecânica e engenharia elétrica, tendo a última destaque para os campos tecnológicos das telecomunicações e computação. A partir de 2011 as áreas tecnológicas com maior destaque foram: Instrumentos, Engenharia civil e Engenharia elétrica.

In [ ]:
import pandas as pd
# Lendo o arquivo despachoxano.csv
ompi_BR_ano=pd.read_csv('ompi_BRxano.csv',encoding='iso-8859-1',delimiter=';')

# substituindo os valores nan por 0 no dataframe
ompi_BR_ano=ompi_BR_ano.fillna(0)
ompi_BR_ano

In [ ]:
# Pegando apenas o ano de 2016
current_Ano = 2016
ompi_BR_ano_1 = (ompi_BR_ano[ompi_BR_ano['Ano'].eq(current_Ano)]
       .sort_values(by='Depósitos', ascending=False).head(10))
ompi_BR_ano_1

In [ ]:
# Vamos adicionar cores de acordo com a área tecnologica (Química, Instrumentos, Engenharia mecânica, Engenharia elétrica, Outras áreas - divisão esta utilizada pela WIPO)
colors = dict(zip(
    ['Química', 'Instrumentos', 'Engenharia mecânica', 'Engenharia elétrica', 'Outras áreas'],
    ['#D84E4E', '#aafbff', '#90d595', '#e48381', '#f7bb5f']
))
group_lk = ompi_BR_ano.set_index('Campo_Tec')['Area_Tec'].to_dict()

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))
ompi_BR_ano_1= ompi_BR_ano_1[::-1]   # flip values from top to bottom
# pass colors values to `color=`
ax.barh(ompi_BR_ano_1['Campo_Tec'], ompi_BR_ano_1['Depósitos'], color=[colors[group_lk[x]] for x in ompi_BR_ano_1['Campo_Tec']])
# iterate over the values to plot labels and values
for i, (Depósitos, Depositante) in enumerate(zip(ompi_BR_ano_1['Depósitos'], ompi_BR_ano_1['Campo_Tec'])):
    #ax.text(Depósitos, i,     Depositante,            ha='right')  
    ax.text(Depósitos, i-.25, group_lk[Depositante],  ha='right')  
    ax.text(Depósitos, i,     Depósitos,           ha='left')   
# Add year right middle portion of canvas
ax.text(1, 0.4, current_Ano, transform=ax.transAxes, size=46, ha='right')

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))
def draw_barchart(ano):
    ompi_BR_ano_1 = ompi_BR_ano[ompi_BR_ano['Ano'].eq(ano)].sort_values(by='Depósitos', ascending=True).tail(10)
    ax.clear()
    ax.barh(ompi_BR_ano_1['Campo_Tec'], ompi_BR_ano_1['Depósitos'], color=[colors[group_lk[x]] for x in ompi_BR_ano_1['Campo_Tec']])
    dx = ompi_BR_ano_1['Depósitos'].max() / 200
    for i, (Depósitos, Depositante) in enumerate(zip(ompi_BR_ano_1['Depósitos'], ompi_BR_ano_1['Campo_Tec'])):
        ax.text(Depósitos-dx, i,     Depositante,           size=14, weight=600, ha='right', va='bottom')
        ax.text(Depósitos-dx, i-.25, group_lk[Depositante], size=12, ha='right', va='baseline')
        ax.text(Depósitos+dx, i,     f'{Depósitos:,.0f}',  size=14, ha='left',  va='center')
    # ... polished styles
    ax.text(1, 0.4, ano, transform=ax.transAxes, color='#777777', size=46, ha='right', weight=800)
    ax.xaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
    ax.xaxis.set_ticks_position('top')
    ax.tick_params(axis='x', colors='#777777', labelsize=12)
    ax.set_yticks([])
    ax.margins(0, 0.01)
    ax.grid(which='major', axis='x', linestyle='-')
    ax.set_axisbelow(True)
    ax.text(0, 1.12, 'Tecnologias de residentes no BR, com maior núm. de pedidos no INPI',
            transform=ax.transAxes, size=24, weight=600, ha='left')
    plt.box(False)
    
draw_barchart(2016)

In [ ]:
# Utilizando o recurso de animação do matplotlib
import matplotlib.animation as animation
from IPython.display import HTML
fig, ax = plt.subplots(figsize=(16, 8))
animator = animation.FuncAnimation(fig, draw_barchart, frames=range(1993, 2017))
HTML(animator.to_jshtml()) 
# or use animator.to_html5_video() or animator.save()

In [ ]:
animator.save('ompi_BR_ano.mp4')

In [ ]:
from google.colab import files
files.download('ompi_BR_ano.mp4')

# **QUARTA ETAPA - Previsão**

Nesta etapa faremos a modelagem com redes neurais para prever o número de depósito de pedidos de patentes, com base no nosso histórico.

Neste última etapa, vamos dividir a modelagem em 4 partes:


1.   Pre-processamento de dados
2.   Construção da RNN
3.   Fazer as previsões e analisar os resultados
4.   Métricas de avaliação


# Parte 1 - Pre-processamento

In [ ]:
# Importando bibliotecas
import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Conv1D, Dropout
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
numpy.random.seed(7)

In [ ]:
# Lendo os dados
dataframe=pandas.read_csv('pub_mes.csv',usecols=[1], skipfooter=3,engine='python',encoding='iso-8859-1',delimiter=';')
dataframe.head(3)

In [ ]:
dataframe.plot()

In [ ]:
# Tamanho do data_set
len(dataframe)

Aqui tratamos os dados que vamos usar. Primeiro ajustamos a escala dos dados para ficarem entre 0 e 1. A seguir, dividimos os dados entre treino (67% primeiros meses) e testes (33% dos meses finais da série)

In [ ]:
#Converte a coluna do dataframe pandas em um vetor numpy
dataset = dataframe.values
dataset = dataset.astype('float32')

look_back = 12

# Divite os dados de treino (2/3) e teste (1/3)
# Note que a divisão não é aleatória, mas sim sequencial
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size-look_back-1:len(dataset),:]

Aqui criamos nossos pares de dados $X$ e $Y$, onde $Y_{t} = X_{t-1}$ (para look_bak=1). Uma outra forma de pensar no valor de $Y$ para um dado $X$ é que ele é o próximo $X$ na série temporal.

In [ ]:
# Recebe uma série e converte em uma matriz com séries deslocadas.
def create_dataset(dataset, look_back=1, std=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back):
        a = dataset[i:(i+look_back), 0]-dataset[i, 0]
        a /= std
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0]-dataset[i + look_back-1, 0])
    return numpy.array(dataX), numpy.array(dataY)

# reshape into X=t and Y=t+1
std = train[:, 0].std()
trainX, trainY = create_dataset(train, look_back, std)
testX, testY = create_dataset(test, look_back, std)
# shape is [samples, time steps, features]

trainX = trainX.reshape(-1, look_back, 1)
testX = testX.reshape(-1, look_back, 1)
trainY = trainY / 30
testY = testY / 30

trainX.shape, testX.shape

# Parte 2 - Construção da RNN

In [ ]:
model = Sequential()
model.add(LSTM(20, input_shape=(look_back, 1), return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.summary()

In [ ]:
# Treinamento
callbacks = [
    ReduceLROnPlateau(patience=10, factor=0.5, verbose=True),
    ModelCheckpoint('best.model.h5', save_best_only=True),
    EarlyStopping(patience=25, verbose=True)
]

history = model.fit(trainX, trainY, epochs=5000, batch_size=24, validation_data=(testX, testY),
                    verbose=0, callbacks=callbacks)

In [ ]:
df_history = pandas.DataFrame(history.history)
ax = df_history[['val_loss', 'loss']].plot(figsize=(10, 5))
df_history['lr'].plot(ax=ax.twinx(), color='gray')

## Parte 3 - Previsões e análise dos resultados

In [ ]:
# Realiza as previsões. Notar que a utilidade de prever trainX é nenhuma. Serve apenas para exibir no gráfico.

trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

Imprime o gráfico da previsão (em vermelho)

In [ ]:
# shift train predictions for plotting
trainPredictPlot = (trainPredict.ravel() * 30) + dataset[look_back:len(trainPredict)+look_back, 0]

# shift test predictions for plotting
testPredictPlot = (testPredict.ravel() * 30) + dataset[len(trainPredict)+(look_back)-1:len(dataset), 0]

# plot baseline and predictions
plt.figure(figsize=(20, 10))
plt.plot(dataset)
plt.plot(look_back+numpy.arange(len(trainPredictPlot)), trainPredictPlot)
plt.plot(look_back+numpy.arange(len(testPredictPlot))+len(trainPredictPlot)-1, testPredictPlot)
plt.show()

## Parte 4 - Métricas de avaliação

In [ ]:
# Calcula os erros de previsão
trainScore = math.sqrt(mean_squared_error(trainY, trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY, testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))